Colab link - https://colab.research.google.com/drive/1kDHeiwHTxeuRxu9rZhN6aTTZrDI4OkS1?usp=sharing

In [255]:

from google.colab import drive
drive.mount('/content/drive')
data_folder = '/content/drive/MyDrive/BA_870/Assignment3/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [256]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# 1983-87
The regression analysis shows that the model has an adjusted R-squared of 0.219, indicating that approximately 21.9% of the variation in stock returns can be explained by the included variables.
Among the variable, debt to equity was significant,and  "smb" (Small Minus Big) and "mktrf" (Market Return) are found to be statistically significant in their impact on stock returns.
The variable "smb" has a coefficient of -0.0092, indicating that for each unit increase in the SMB factor, there is a decrease of approximately 0.0092 in stock returns. This suggests that smaller companies tend to have lower returns compared to larger companies.
The variable "mktrf" has a coefficient of -0.0555, implying that for each unit increase in the market return, there is a decrease of approximately 0.0555 in stock returns. This suggests that market returns have a negative effect on individual stock returns.

the g_sector_15, g_sector_20, g_sector_60 are also found to be the significant.

so these all significant variables are considered to be the most importtant variables in predicting the stock market return during the crash of 1983-87 

## step 1

### regression with financial ratios analysis

From the regression results, the adjusted R-squared value is 0.024, indicating that the model explains approximately 2.4% of the variability in the return (RET). Among the variables included in the model, two variables, ln_at (log of total assets) and Net_Profit_Margin, show statistically significant results.
For every one unit increase in ln_at, the return is expected to increase by approximately 0.0028 units. Similarly, for every one unit increase in Net_Profit_Margin, the return is expected to increase by approximately 0.0090 units.

In [257]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from scipy.stats.mstats import winsorize



ret_data_1983_87 = pd.read_csv(data_folder + 'CRSP-1983-1987.csv')

crash_data = ret_data_1983_87[ret_data_1983_87["date"] > '1987-08-31']

crash_return = crash_data.groupby('TICKER')['RET'].mean()

Compustat_1986 = pd.read_csv(data_folder +'Compustat-1986.csv')

#merging both the dataframe
merged_df_83_87 = pd.merge(Compustat_1986, crash_return, left_on='tic', right_on='TICKER')
merged_df_83_87.sort_values(by='tic')

###missing data for act
missing_data = merged_df_83_87[merged_df_83_87['act'].isnull()]
X = np.log(missing_data[['at']] + 1)
y = np.log(missing_data['act'].fillna(1))

# Filter out rows with zero or negative values
valid_idx = (X >= 0).all(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Fit the linear regression model
reg = LinearRegression().fit(X, y)

# Select the subset of the DataFrame that has missing values for 'act'
missing_data = merged_df_83_87[merged_df_83_87['act'].isnull()]
X_test = np.log(missing_data[['at']] + 1)

# Fill missing 'act' values with the predicted values
y_pred = np.exp(reg.predict(X_test))
merged_df_83_87.loc[X_test.index, 'act'] = y_pred

##missing data for lct

missing_data = merged_df_83_87[merged_df_83_87['lct'].isnull()]
X = np.log(missing_data[['lt']] + 1)
y = np.log(missing_data['lct'].fillna(1))

# Filter out rows with zero or negative values
valid_idx = (X >= 0).all(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Fit the linear regression model
reg = LinearRegression().fit(X, y)

# Select the subset of the DataFrame that has missing values for 'lct'
missing_data = merged_df_83_87[merged_df_83_87['lct'].isnull()]
X_test = np.log(missing_data[['lt']] + 1)

# Fill missing 'lct' values with the predicted values
y_pred = np.exp(reg.predict(X_test))
merged_df_83_87.loc[X_test.index, 'lct'] = y_pred





#using log for the total assets, current assets and curent liability

merged_df_83_87['ln_at'] = np.log(merged_df_83_87['at'] + 1)
merged_df_83_87['ln_at']

merged_df_83_87['ln_act'] = np.log(merged_df_83_87['act'] + 1)
merged_df_83_87['ln_act']

merged_df_83_87['ln_lct'] = np.log(merged_df_83_87['lct'] + 1)
merged_df_83_87['ln_lct']


# Calculate Current Ratio
merged_df_83_87['current_ratio'] = merged_df_83_87['act'] / merged_df_83_87['lct']

# Calculate Debt to Equity Ratio
merged_df_83_87['debt_to_equity_ratio'] = merged_df_83_87['dltt'] / (merged_df_83_87['at'] - merged_df_83_87['lt'])

# Calculate Return on Equity (ROE)
merged_df_83_87['return_on_equity'] = merged_df_83_87['ni'] / (merged_df_83_87['at'] - merged_df_83_87['lt'])

# Calculate Asset Turnover Ratio
merged_df_83_87['asset_turnover_ratio'] = merged_df_83_87['sale'] / merged_df_83_87['at']

# Debt-to-Assets Ratio
merged_df_83_87['Debt_to_Assets_Ratio'] = merged_df_83_87['dltt'] / merged_df_83_87['at']

# Net Profit Margin
merged_df_83_87['Net_Profit_Margin'] = merged_df_83_87['ni'] / merged_df_83_87['sale']



#droping NA
merged_df_83_87.dropna(subset=['current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio',  'RET'], inplace=True)



#winsorizing the data
# create a list of ratio columns to use in the regression
ratio_cols = ['current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio']

limits = [0.01, 0.01]

# loop through all columns except gvkey
for col in merged_df_83_87.columns[-10:]:
    merged_df_83_87[col] = winsorize(merged_df_83_87[col], limits)

# print the resulting DataFrame
# print(merged_df_83_87)


#regressing the ratios


# create a list of ratio columns to use in the regression
ratio_cols = ['ln_at','current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio', 'Debt_to_Assets_Ratio','Net_Profit_Margin' ]
# define the dependent variable
y = merged_df_83_87['RET']

# define the independent variables
X = sm.add_constant(merged_df_83_87[ratio_cols])

# fit the linear regression model
model = sm.OLS(y, X).fit()

# print the regression summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     6.760
Date:                Sat, 06 May 2023   Prob (F-statistic):           6.31e-08
Time:                        07:48:53   Log-Likelihood:                 2136.1
No. Observations:                1667   AIC:                            -4256.
Df Residuals:                    1659   BIC:                            -4213.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0832 

### for second variable, regression with smb and hml
The regression analysis indicates an adjusted R-squared value of 0.185, indicating that approximately 18.5% of the variability in the return (RET) can be explained by the model. Among the variables included in the model, two variables, mktrf (market risk factor) and smb (size factor), show statistically significant results.

The market risk factor (mktrf) has a significant negative coefficient of -0.0574. This implies that for every one unit increase in mktrf, the return is expected to decrease by approximately 0.0574 units. Similarly, the size factor (smb) has a significant negative coefficient of -0.0120, suggesting that for every one unit increase in smb, the return is expected to decrease by approximately 0.0120 units.



#### cleaning and merging

In [258]:
data_folder_2 = '/content/drive/MyDrive/BA_870/Data/' 

In [259]:
risk_1983_87 = pd.read_csv(data_folder_2 + 'FF-Output-1983_87.csv')
risk_1983_87.head(5)

,index,Ticker,const,mktrf,smb,hml
0,0,AA,0.002627,0.908593,0.823196,0.120751
1,1,AAA,0.006727,1.323926,0.179244,-0.323982
2,2,AAC,0.005391,1.038395,1.516546,-2.195825
3,3,AAGIY,-0.003048,0.659641,1.220527,1.087526
4,4,AAL,-0.011023,0.858837,0.790875,0.416452


In [260]:
risk_1983_87.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2982 entries, 0 to 2981
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   2982 non-null   int64  
 1   Ticker  2982 non-null   object 
 2   const   2982 non-null   float64
 3   mktrf   2982 non-null   float64
 4   smb     2982 non-null   float64
 5   hml     2982 non-null   float64
dtypes: float64(4), int64(1), object(1)
memory usage: 139.9+ KB


In [261]:
merged_df_83_87 = pd.merge(merged_df_83_87, risk_1983_87, left_on='tic', right_on='Ticker')
merged_df_83_87.head(5)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,curcd,act,...,return_on_equity,asset_turnover_ratio,Debt_to_Assets_Ratio,Net_Profit_Margin,index,Ticker,const,mktrf,smb,hml
0,1004,1987-05-31,1986,INDL,C,D,STD,AIR,USD,168.950,...,0.114370,1.268411,0.053073,0.051514,90,AIR,0.020442,1.143620,0.478206,-0.238106
1,1016,1986-10-31,1986,INDL,C,D,STD,AECE,USD,18.715,...,0.080062,2.344443,0.192048,0.015301,50,AECE,0.020248,1.184693,1.976234,0.274068
2,1017,1987-02-28,1986,INDL,C,D,STD,AELNA,USD,52.080,...,0.059649,1.198293,0.088118,0.030642,53,AELNA,0.002670,0.876571,2.101294,-0.245387
3,1019,1986-12-31,1986,INDL,C,D,STD,AFAP,USD,4.511,...,0.187059,0.950141,0.000000,0.066999,59,AFAP,0.006200,0.287011,0.257466,0.109483
4,1045,1986-12-31,1986,INDL,C,D,STD,AAL,USD,2109.407,...,0.111257,0.799442,0.320409,0.046382,4,AAL,-0.011023,0.858837,0.790875,0.416452


In [262]:
merged_df_83_87.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 0 to 878
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey                 879 non-null    int64  
 1   datadate              879 non-null    object 
 2   fyear                 879 non-null    int64  
 3   indfmt                879 non-null    object 
 4   consol                879 non-null    object 
 5   popsrc                879 non-null    object 
 6   datafmt               879 non-null    object 
 7   tic                   879 non-null    object 
 8   curcd                 879 non-null    object 
 9   act                   879 non-null    float64
 10  at                    879 non-null    float64
 11  che                   879 non-null    float64
 12  cogs                  878 non-null    float64
 13  dltt                  879 non-null    float64
 14  lct                   879 non-null    float64
 15  lt                    8

#### regression

In [263]:
import pandas as pd
import statsmodels.api as sm



# construct the independent variables using the beta coefficients and the variables in merged_df_83_87
mktrf = merged_df_83_87['mktrf']
smb = merged_df_83_87['smb']
hml = merged_df_83_87['hml']
const = merged_df_83_87['const']

# construct the dependent variable
RET = merged_df_83_87['RET']

# create a regression model with RET as the dependent variable and mktrf, smb, hml as the independent variables
X = sm.add_constant(pd.concat([mktrf, smb, hml], axis=1))
model = sm.OLS(RET, X)

# fit the model
results = model.fit()

# print the regression summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     67.46
Date:                Sat, 06 May 2023   Prob (F-statistic):           3.01e-39
Time:                        07:48:53   Log-Likelihood:                 1369.1
No. Observations:                 879   AIC:                            -2730.
Df Residuals:                     875   BIC:                            -2711.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0013      0.005     -0.263      0.7

### regression with industry effects

The regression results indicate that the model has an adjusted R-squared of 0.057, implying that approximately 5.7% of the variability in the return (RET) can be explained by the variables included in the model. Among the variables, several sectors (gsector) show statistically significant relationships with the return.The coefficients of the gsector variables suggest that being in certain sectors has a positive impact on the return. Specifically, gsector_15.0, gsector_20.0, gsector_30.0, gsector_35.0, gsector_55.0, and gsector_60.0 have statistically significant coefficients. This means that being in these sectors is associated with an increase in the return. The magnitudes of these effects can be seen from the coefficients themselves (e.g., gsector_15.0 has a coefficient of 0.0449).

#### cleaning and merging

In [264]:
industry_1983_87 = pd.read_csv(data_folder_2 + 'GSECTOR-1986.csv')
industry_1983_87.head(5)

,gvkey,fyear,tic,gsector
0,1003,1986,ANTQ,25.0
1,1004,1986,AIR,20.0
2,1009,1986,ABSI.1,15.0
3,1010,1986,4165A,20.0
4,1011,1986,ACSE,25.0


In [265]:
industry_1983_87.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7219 entries, 0 to 7218
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   gvkey    7219 non-null   int64  
 1   fyear    7219 non-null   int64  
 2   tic      7219 non-null   object 
 3   gsector  7210 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 225.7+ KB


In [266]:
len(industry_1983_87.gsector.unique())

12

In [267]:
merged_df_83_87 = pd.merge(merged_df_83_87, industry_1983_87, left_on='tic', right_on='tic')
merged_df_83_87.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 0 to 878
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               879 non-null    int64  
 1   datadate              879 non-null    object 
 2   fyear_x               879 non-null    int64  
 3   indfmt                879 non-null    object 
 4   consol                879 non-null    object 
 5   popsrc                879 non-null    object 
 6   datafmt               879 non-null    object 
 7   tic                   879 non-null    object 
 8   curcd                 879 non-null    object 
 9   act                   879 non-null    float64
 10  at                    879 non-null    float64
 11  che                   879 non-null    float64
 12  cogs                  878 non-null    float64
 13  dltt                  879 non-null    float64
 14  lct                   879 non-null    float64
 15  lt                    8

#### regression

In [268]:
merged_df_83_87.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 0 to 878
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               879 non-null    int64  
 1   datadate              879 non-null    object 
 2   fyear_x               879 non-null    int64  
 3   indfmt                879 non-null    object 
 4   consol                879 non-null    object 
 5   popsrc                879 non-null    object 
 6   datafmt               879 non-null    object 
 7   tic                   879 non-null    object 
 8   curcd                 879 non-null    object 
 9   act                   879 non-null    float64
 10  at                    879 non-null    float64
 11  che                   879 non-null    float64
 12  cogs                  878 non-null    float64
 13  dltt                  879 non-null    float64
 14  lct                   879 non-null    float64
 15  lt                    8

In [269]:
import pandas as pd
import statsmodels.api as sm

# create dummy variables for gsector column
gsector_dummies = pd.get_dummies(merged_df_83_87['gsector'], prefix='gsector')

# concatenate the dummy variables with the original data frame
merged_df_83_87 = pd.concat([merged_df_83_87, gsector_dummies], axis=1)

# set RET as the dependent variable
y = merged_df_83_87['RET']

merged_df_83_87 = merged_df_83_87.drop(columns=['gsector_10.0'])        #removing gsector 10
# print(merged_df_83_87.info())
# set the independent variables to the dummy variables for gsector
X = merged_df_83_87.filter(regex='^gsector_')

# add a constant to the independent variables
X = sm.add_constant(X)

# create the OLS regression model
model = sm.OLS(y, X)

# fit the model
results = model.fit()

# print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     6.297
Date:                Sat, 06 May 2023   Prob (F-statistic):           2.27e-09
Time:                        07:48:53   Log-Likelihood:                 1308.4
No. Observations:                 879   AIC:                            -2595.
Df Residuals:                     868   BIC:                            -2542.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0878      0.008    -11.525   

In [270]:
merged_df_83_87.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 0 to 878
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               879 non-null    int64  
 1   datadate              879 non-null    object 
 2   fyear_x               879 non-null    int64  
 3   indfmt                879 non-null    object 
 4   consol                879 non-null    object 
 5   popsrc                879 non-null    object 
 6   datafmt               879 non-null    object 
 7   tic                   879 non-null    object 
 8   curcd                 879 non-null    object 
 9   act                   879 non-null    float64
 10  at                    879 non-null    float64
 11  che                   879 non-null    float64
 12  cogs                  878 non-null    float64
 13  dltt                  879 non-null    float64
 14  lct                   879 non-null    float64
 15  lt                    8

## step 2 

### Regression with industry effect and financial ratios

The regression results indicate that the model has an adjusted R-squared of 0.082, implying that approximately 8.2% of the variability in the return (RET) can be explained by the variables included in the model. Among the variables, several sectors (gsector) show statistically significant relationships with the return.The coefficients of the gsector variables suggest that being in certain sectors has a positive impact on the return. Specifically, gsector_15.0, gsector_20.0, gsector_30.0, gsector_35.0, gsector_55.0, and gsector_60.0 have statistically significant coefficients. This means that being in these sectors is associated with an increase in the return. The magnitudes of these effects can be seen from the coefficients themselves (e.g., gsector_15.0 has a coefficient of 0.0407).

Additionally, some of the ratios, such as debt_to_equity_ratio and Net_Profit_Margin, show statistically significant relationships with the return. The coefficient for debt_to_equity_ratio suggests that an increase in this ratio is associated with a decrease in the return. On the other hand, an increase in Net_Profit_Margin is associated with an increase in the return.


In [271]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_83_87 is the DataFrame you provided

# Set the dependent variable
y = merged_df_83_87['RET']

# Filter the gsector columns with numbers behind them
gsector_columns = [col for col in merged_df_83_87.columns if col.startswith('gsector_')]

ratios = ['ln_at','current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio', 'Debt_to_Assets_Ratio','Net_Profit_Margin' ]
# Set the independent variables
X = merged_df_83_87[gsector_columns + ratios]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.100
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     5.642
Date:                Sat, 06 May 2023   Prob (F-statistic):           2.96e-12
Time:                        07:48:53   Log-Likelihood:                 1324.1
No. Observations:                 879   AIC:                            -2612.
Df Residuals:                     861   BIC:                            -2526.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0867 

### regression with industry effect and smb/hml
The regression results indicate that the model has an adjusted R-squared of 0.208, implying that approximately 20.8% of the variability in the return (RET) can be explained by the variables included in the model. Among the variables, several sectors (gsector) and factor returns (smb, hml, mktrf) show statistically significant relationships with the return.The coefficients of the gsector variables suggest that being in certain sectors has a positive impact on the return. Specifically, gsector_15.0, gsector_20.0, gsector_30.0, and gsector_60.0 have statistically significant coefficients. This means that being in these sectors is associated with an increase in the return. The magnitudes of these effects can be seen from the coefficients themselves (e.g., gsector_15.0 has a coefficient of 0.0400).

Additionally, the factors smb (Small Minus Big) and mktrf (Market Return minus Risk-Free Rate) show statistically significant relationships with the return. The coefficient for smb suggests that an increase in this factor return is associated with a decrease in the return. On the other hand, there is no statistically significant relationship between the return and hml. The coefficient for mktrf suggests that an increase in the market return relative to the risk-free rate is associated with a decrease in the return.



In [272]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_83_87 is the DataFrame you provided

# Set the dependent variable
y = merged_df_83_87['RET']

# Filter the gsector columns with numbers behind them
gsector_columns = [col for col in merged_df_83_87.columns if col.startswith('gsector_')]

smb_hml = ['smb', 'hml', 'mktrf']
# Set the independent variables
X = merged_df_83_87[gsector_columns + smb_hml]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     18.78
Date:                Sat, 06 May 2023   Prob (F-statistic):           5.78e-39
Time:                        07:48:53   Log-Likelihood:                 1386.9
No. Observations:                 879   AIC:                            -2746.
Df Residuals:                     865   BIC:                            -2679.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0216      0.009     -2.494   

### Regression with financial ratios and smsb_hml

The regression analysis was performed to examine the relationship between the return (RET) and several independent variables. The model has an R-squared value of 0.210, and adj R-squared 0.201 indicating that approximately 20.1% of the variation in the return can be explained by the included variables.

Among the independent variables, the debt-to-equity ratio demonstrates a statistically significant relationship with the return (p-value = 0.003). The coefficient estimate for the debt-to-equity ratio is -0.0076, suggesting that a decrease in the debt-to-equity ratio is associated with an increase in the return. This implies that companies with lower levels of debt in relation to their equity tend to have higher returns.

Furthermore, the asset turnover ratio also shows a statistically significant relationship with the return (p-value = 0.032). The coefficient estimate for the asset turnover ratio is 0.0050, indicating that a higher asset turnover ratio is associated with a higher return. This suggests that companies that efficiently utilize their assets to generate revenue tend to have higher returns. Also, smb and mkrtf shows a significant results.
Among the other variables, "smb" (Small Minus Big) and "mktrf" (Market Return) are found to be statistically significant as well.

In [273]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_83_87 is the DataFrame you provided

# Set the dependent variable
y = merged_df_83_87['RET']

# Filter the gsector columns with numbers behind them
# gsector_columns = [col for col in merged_df_83_87.columns if col.startswith('gsector_')]

# Set the independent variables
X = merged_df_83_87[ratios + smb_hml]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.210
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     23.12
Date:                Sat, 06 May 2023   Prob (F-statistic):           9.61e-39
Time:                        07:48:53   Log-Likelihood:                 1381.4
No. Observations:                 879   AIC:                            -2741.
Df Residuals:                     868   BIC:                            -2688.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0144 

#### All 3 together

The regression analysis shows that the model has an adjusted R-squared of 0.219, indicating that approximately 21.9% of the variation in stock returns can be explained by the included variables.
Among the variable, debt to equity was significant,and  "smb" (Small Minus Big) and "mktrf" (Market Return) are found to be statistically significant in their impact on stock returns.
The variable "smb" has a coefficient of -0.0092, indicating that for each unit increase in the SMB factor, there is a decrease of approximately 0.0092 in stock returns. This suggests that smaller companies tend to have lower returns compared to larger companies.
The variable "mktrf" has a coefficient of -0.0555, implying that for each unit increase in the market return, there is a decrease of approximately 0.0555 in stock returns. This suggests that market returns have a negative effect on individual stock returns.

the g_sector_15, g_sector_20, g_sector_60 are found to be the significant.

so these all significant variables are found to be the most important in the crash of 1983 to 1987 



In [274]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_83_87 is the DataFrame you provided

# Set the dependent variable
y = merged_df_83_87['RET']

# Filter the gsector columns with numbers behind them
# gsector_columns = [col for col in merged_df_83_87.columns if col.startswith('gsector_')]

# Set the independent variables
X = merged_df_83_87[ratios + smb_hml + gsector_columns]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                     13.28
Date:                Sat, 06 May 2023   Prob (F-statistic):           2.06e-38
Time:                        07:48:53   Log-Likelihood:                 1396.2
No. Observations:                 879   AIC:                            -2750.
Df Residuals:                     858   BIC:                            -2650.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0279 

# 1997 - 2001

he updated OLS regression results show that the model has an R-squared value of 0.481, indicating that approximately 48.1% of the variance in stock returns can be explained by the included variables. The adjusted R-squared value is 0.470.

Among the included variables, the variable "return_on_equity" still has a significant positive coefficient, indicating that companies with higher return on equity tend to have higher stock returns. Similarly, the variables "hml" (high-minus-low) and "mktrf" (market factor) continue to have significant positive coefficients, suggesting that a higher difference between value and growth stock returns and a positive market return are associated with higher stock returns. The g_sector_20 was found to be significant as well.

so for that industry, significant ratios and significant market risk ecposure could have helped to predict the return on stocks during the crash of 1997-2001

## step 1

### regression with financial ratios analysis

The regression analysis results indicate that the model has a coefficient of determination (R-squared) of 0.156, implying that approximately 15.6% of the variance in stock returns can be explained by the included variables.
Firstly, the variable "current_ratio" exhibits a statistically significant negative relationship with stock returns. A unit increase in the current ratio is associated with a decrease in stock returns of approximately 0.0046.Second, the variable "debt_to_equity_ratio" shows a statistically significant positive relationship with stock returns. A unit increase in the debt-to-equity ratio corresponds to an increase in stock returns of approximately 0.0061. This indicates that companies with higher levels of debt relative to equity tend to have higher stock returns.Additionally, the variables "return_on_equity," "asset_turnover_ratio," "Debt_to_Assets_Ratio," and "Net_Profit_Margin" also exhibit statistically significant relationships with stock returns. 

In [275]:
ret_data_1997_01 = pd.read_csv(data_folder + 'CRSP-1997-2001.csv')

crash_data = ret_data_1997_01[(ret_data_1997_01["date"] <= '2001-03-31') & (ret_data_1997_01["date"] >= '2000-09-01')]

crash_return = crash_data.groupby('TICKER')['RET'].mean()

Compustat_2000 = pd.read_csv(data_folder +'Compustat-2000.csv')

#merging both the dataframe
merged_df_97_01 = pd.merge(Compustat_2000, crash_return, left_on='tic', right_on='TICKER')
merged_df_97_01


###missing data for act
missing_data = merged_df_97_01[merged_df_97_01['act'].isnull()]
X = np.log(missing_data[['at']] + 1)
y = np.log(missing_data['act'].fillna(1))

# Filter out rows with zero or negative values
valid_idx = (X >= 0).all(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Fit the linear regression model
reg = LinearRegression().fit(X, y)

# Select the subset of the DataFrame that has missing values for 'act'
missing_data = merged_df_97_01[merged_df_97_01['act'].isnull()]
X_test = np.log(missing_data[['at']] + 1)

# Fill missing 'act' values with the predicted values
y_pred = np.exp(reg.predict(X_test))
merged_df_97_01.loc[X_test.index, 'act'] = y_pred

##missing data for lct

missing_data = merged_df_97_01[merged_df_97_01['lct'].isnull()]
X = np.log(missing_data[['lt']] + 1)
y = np.log(missing_data['lct'].fillna(1))

# Filter out rows with zero or negative values
valid_idx = (X >= 0).all(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Fit the linear regression model
reg = LinearRegression().fit(X, y)

# Select the subset of the DataFrame that has missing values for 'lct'
missing_data = merged_df_97_01[merged_df_97_01['lct'].isnull()]
X_test = np.log(missing_data[['lt']] + 1)

# Fill missing 'lct' values with the predicted values
y_pred = np.exp(reg.predict(X_test))
merged_df_97_01.loc[X_test.index, 'lct'] = y_pred





#using log for the total assets, current assets and curent liability

merged_df_97_01['ln_at'] = np.log(merged_df_97_01['at'] + 1)
merged_df_97_01['ln_at']

merged_df_97_01['ln_act'] = np.log(merged_df_97_01['act'] + 1)
merged_df_97_01['ln_act']

merged_df_97_01['ln_lct'] = np.log(merged_df_97_01['lct'] + 1)
merged_df_97_01['ln_lct']


# Calculate Current Ratio
merged_df_97_01['current_ratio'] = merged_df_97_01['act'] / merged_df_97_01['lct']

# Calculate Debt to Equity Ratio
merged_df_97_01['debt_to_equity_ratio'] = merged_df_97_01['dltt'] / (merged_df_97_01['at'] - merged_df_97_01['lt'])

# Calculate Return on Equity (ROE)
merged_df_97_01['return_on_equity'] = merged_df_97_01['ni'] / (merged_df_97_01['at'] - merged_df_97_01['lt'])

# Calculate Asset Turnover Ratio
merged_df_97_01['asset_turnover_ratio'] = merged_df_97_01['sale'] / merged_df_97_01['at']

# Debt-to-Assets Ratio
merged_df_97_01['Debt_to_Assets_Ratio'] = merged_df_97_01['dltt'] / merged_df_97_01['at']

# Net Profit Margin
merged_df_97_01['Net_Profit_Margin'] = merged_df_97_01['ni'] / merged_df_97_01['sale']



#droping NA
merged_df_97_01.dropna(subset=['current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio',  'RET'], inplace=True)



#winsorizing the data
# create a list of ratio columns to use in the regression
ratio_cols = ['current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio']

limits = [0.01, 0.01]

# loop through all columns except gvkey
for col in merged_df_97_01.columns[-10:]:
    merged_df_97_01[col] = winsorize(merged_df_97_01[col], limits)

merged_df_97_01['ln_at'] = winsorize(merged_df_97_01['ln_at'], limits)

# print the resulting DataFrame
# print(merged_df_97_01)


#regressing the ratios


# create a list of ratio columns to use in the regression
ratio_cols = ['ln_at', 'current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio', 'Debt_to_Assets_Ratio','Net_Profit_Margin' ]
# define the dependent variable
y = merged_df_97_01['RET']

# define the independent variables
X = sm.add_constant(merged_df_97_01[ratio_cols])

# fit the linear regression model
model = sm.OLS(y, X).fit()

# print the regression summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     96.67
Date:                Sat, 06 May 2023   Prob (F-statistic):          5.79e-130
Time:                        07:48:54   Log-Likelihood:                 4397.0
No. Observations:                3678   AIC:                            -8778.
Df Residuals:                    3670   BIC:                            -8728.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0217 

### for second variable, regression with smb and hml

#### cleaning and merging

In [276]:
data_folder_2 = '/content/drive/MyDrive/BA_870/Data/' 

In [277]:
risk_1997_01 = pd.read_csv(data_folder_2 + 'FF-Output-1997_01.csv')
risk_1997_01.head(5)

,Ticker,const,mktrf,smb,hml
0,AA,0.005322,1.477028,0.363017,0.710288
1,AABC,-0.001837,0.656875,0.437663,0.871911
2,AACE,-0.002621,0.888312,0.475687,0.674179
3,AAII,0.018905,1.121662,0.901589,0.240981
4,AALA,0.081005,-1.780657,-0.071121,-0.963159


In [278]:
risk_1997_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3928 entries, 0 to 3927
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Ticker  3928 non-null   object 
 1   const   3928 non-null   float64
 2   mktrf   3928 non-null   float64
 3   smb     3928 non-null   float64
 4   hml     3928 non-null   float64
dtypes: float64(4), object(1)
memory usage: 153.6+ KB


In [279]:
merged_df_97_01 = pd.merge(merged_df_97_01, risk_1997_01, left_on='tic', right_on='Ticker')
merged_df_97_01.head(5)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,curcd,act,...,debt_to_equity_ratio,return_on_equity,asset_turnover_ratio,Debt_to_Assets_Ratio,Net_Profit_Margin,Ticker,const,mktrf,smb,hml
0,1004,2001-05-31,2000,INDL,C,D,STD,AIR,USD,485.856,...,0.529044,0.054469,1.245637,0.256445,0.021196,AIR,-0.017474,1.051142,1.542077,1.187563
1,1062,2000-11-30,2000,INDL,C,D,STD,ASA,USD,1.000,...,0.000000,0.031631,0.045784,0.000000,0.663393,ASA,-0.008481,0.589249,0.525356,0.268877
2,1072,2001-03-31,2000,INDL,C,D,STD,AVX,USD,1355.286,...,0.009117,0.377092,1.383542,0.007279,0.217604,AVX,0.022348,0.785441,0.125677,-0.596908
3,1075,2000-12-31,2000,INDL,C,D,STD,PNW,USD,793.913,...,0.820528,0.126886,0.516170,0.273471,0.081929,PNW,-0.000949,0.348717,0.130700,0.973273
4,1078,2000-12-31,2000,INDL,C,D,STD,ABT,USD,7376.241,...,0.125584,0.325050,0.899410,0.070428,0.202677,ABT,0.009124,0.478550,-0.353840,0.245987


In [280]:
merged_df_97_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 1970
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey                 1971 non-null   int64  
 1   datadate              1971 non-null   object 
 2   fyear                 1971 non-null   int64  
 3   indfmt                1971 non-null   object 
 4   consol                1971 non-null   object 
 5   popsrc                1971 non-null   object 
 6   datafmt               1971 non-null   object 
 7   tic                   1971 non-null   object 
 8   curcd                 1971 non-null   object 
 9   act                   1971 non-null   float64
 10  at                    1971 non-null   float64
 11  che                   1971 non-null   float64
 12  cogs                  1971 non-null   float64
 13  dltt                  1971 non-null   float64
 14  lct                   1971 non-null   float64
 15  lt                   

#### regression

regression results indicate that the model has an R-squared of 0.416, suggesting that approximately 41.6% of the variance in stock returns can be explained by the included variables. The adjusted R-squared value is also 0.415.
Among the variables, three variables show significant relationships with stock returns. The variable "mktrf" (market risk premium) has a negative coefficient of -0.0304, indicating that an increase in the market risk premium is associated with a decrease in stock returns. The variable "smb" (size factor) has a negative coefficient of -0.0032, suggesting that an increase in the size factor is associated with a decrease in stock returns.The variable "hml" (value factor) has a coefficient of 0.0390, indicating that an increase in the value factor is associated with an increase in stock returns.

In [281]:
import pandas as pd
import statsmodels.api as sm



# construct the independent variables using the beta coefficients and the variables in merged_df_97_01
mktrf = merged_df_97_01['mktrf']
smb = merged_df_97_01['smb']
hml = merged_df_97_01['hml']
const = merged_df_97_01['const']

# construct the dependent variable
RET = merged_df_97_01['RET']

# create a regression model with RET as the dependent variable and mktrf, smb, hml as the independent variables
X = sm.add_constant(pd.concat([mktrf, smb, hml], axis=1))
model = sm.OLS(RET, X)

# fit the model
results = model.fit()

# print the regression summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.415
Method:                 Least Squares   F-statistic:                     467.4
Date:                Sat, 06 May 2023   Prob (F-statistic):          2.99e-229
Time:                        07:48:54   Log-Likelihood:                 3352.9
No. Observations:                1971   AIC:                            -6698.
Df Residuals:                    1967   BIC:                            -6675.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0123      0.002      6.452      0.0

### regression with industry effects

the model has an R-squared of 0.162, indicating that approximately 16.2% of the variance in stock returns can be explained by the included variables. The adjusted R-squared value is 0.153. Among the included variables, several sectors show significant relationships with stock returns. The variables "gsector_20.0" (sector 20), "gsector_35.0" (sector 35), and "gsector_45.0" (sector 45) have negative coefficients, suggesting that belonging to these sectors is associated with a decrease in stock returns. On the other hand, the variable "gsector_50.0" (sector 50) has a coefficient with a p-value of 0.094, which is close to the significance threshold of 0.05. This suggests that belonging to sector 50 may have some influence on stock returns, but further investigation is warranted.

#### cleaning and merging

In [282]:
industry_1983_87 = pd.read_csv(data_folder_2 + 'GSECTOR-1986.csv')
industry_1983_87.head(5)

,gvkey,fyear,tic,gsector
0,1003,1986,ANTQ,25.0
1,1004,1986,AIR,20.0
2,1009,1986,ABSI.1,15.0
3,1010,1986,4165A,20.0
4,1011,1986,ACSE,25.0


In [283]:
industry_1983_87.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7219 entries, 0 to 7218
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   gvkey    7219 non-null   int64  
 1   fyear    7219 non-null   int64  
 2   tic      7219 non-null   object 
 3   gsector  7210 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 225.7+ KB


In [284]:
len(industry_1983_87.gsector.unique())

12

In [285]:
merged_df_97_01 = pd.merge(merged_df_97_01, industry_1983_87, left_on='tic', right_on='tic')
merged_df_97_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 939 entries, 0 to 938
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               939 non-null    int64  
 1   datadate              939 non-null    object 
 2   fyear_x               939 non-null    int64  
 3   indfmt                939 non-null    object 
 4   consol                939 non-null    object 
 5   popsrc                939 non-null    object 
 6   datafmt               939 non-null    object 
 7   tic                   939 non-null    object 
 8   curcd                 939 non-null    object 
 9   act                   939 non-null    float64
 10  at                    939 non-null    float64
 11  che                   939 non-null    float64
 12  cogs                  939 non-null    float64
 13  dltt                  939 non-null    float64
 14  lct                   939 non-null    float64
 15  lt                    9

#### regression

In [286]:
merged_df_97_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 939 entries, 0 to 938
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               939 non-null    int64  
 1   datadate              939 non-null    object 
 2   fyear_x               939 non-null    int64  
 3   indfmt                939 non-null    object 
 4   consol                939 non-null    object 
 5   popsrc                939 non-null    object 
 6   datafmt               939 non-null    object 
 7   tic                   939 non-null    object 
 8   curcd                 939 non-null    object 
 9   act                   939 non-null    float64
 10  at                    939 non-null    float64
 11  che                   939 non-null    float64
 12  cogs                  939 non-null    float64
 13  dltt                  939 non-null    float64
 14  lct                   939 non-null    float64
 15  lt                    9

In [287]:
import pandas as pd
import statsmodels.api as sm

# create dummy variables for gsector column
gsector_dummies = pd.get_dummies(merged_df_97_01['gsector'], prefix='gsector')

# concatenate the dummy variables with the original data frame
merged_df_97_01 = pd.concat([merged_df_97_01, gsector_dummies], axis=1)

# set RET as the dependent variable
y = merged_df_97_01['RET']

merged_df_97_01 = merged_df_97_01.drop(columns=['gsector_10.0'])        #removing gsector 10
# print(merged_df_97_01.info())
# set the independent variables to the dummy variables for gsector
X = merged_df_97_01.filter(regex='^gsector_')

# add a constant to the independent variables
X = sm.add_constant(X)

# create the OLS regression model
model = sm.OLS(y, X)

# fit the model
results = model.fit()

# print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     17.92
Date:                Sat, 06 May 2023   Prob (F-statistic):           3.78e-30
Time:                        07:48:54   Log-Likelihood:                 1519.9
No. Observations:                 939   AIC:                            -3018.
Df Residuals:                     928   BIC:                            -2964.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0171      0.007      2.603   

## step 2 

### Regression with industry effect and financial ratios

egression results indicate that the model has an R-squared value of 0.216, suggesting that approximately 21.6% of the variance in stock returns can be explained by the included variables. The adjusted R-squared value is 0.201.
Among the included variables, several sectors show significant relationships with stock returns.The variables "gsector_20.0" (sector 20) and "gsector_45.0" (sector 45) have negative coefficients, indicating that belonging to these sectors is associated with a decrease in stock returns. the other variables, "current_ratio" and "return_on_equity" have negative coefficients, suggesting that higher current ratios and higher return on equity are associated with lower stock returns.vOn the other hand, "ln_at" (natural logarithm of total assets) has a positive coefficient, indicating that larger total assets may be associated with higher stock returns.

In [288]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_97_01 is the DataFrame you provided

# Set the dependent variable
y = merged_df_97_01['RET']

# Filter the gsector columns with numbers behind them
gsector_columns = [col for col in merged_df_97_01.columns if col.startswith('gsector_')]

ratios = ['ln_at','current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio', 'Debt_to_Assets_Ratio','Net_Profit_Margin' ]
# Set the independent variables
X = merged_df_97_01[gsector_columns + ratios]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.216
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     14.89
Date:                Sat, 06 May 2023   Prob (F-statistic):           2.06e-38
Time:                        07:48:55   Log-Likelihood:                 1551.0
No. Observations:                 939   AIC:                            -3066.
Df Residuals:                     921   BIC:                            -2979.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0063 

### regression with industry effect and smb/hml
The OLS regression results show that the model has an R-squared value of 0.457, indicating that approximately 45.7% of the variance in stock returns can be explained by the included variables. The adjusted R-squared value is 0.450.

Among the included variables, the sectors "gsector_15.0" and "gsector_20.0" exhibit negative coefficients, indicating that belonging to these sectors is associated with lower stock returns.

The variables "hml" (high-minus-low), which represents the return difference between value and growth stocks, and "mktrf" (market factor), which captures the overall market return, both have significant positive coefficients. This suggests that a higher difference between value and growth stock returns and a positive market return are associated with higher stock returns.

In [289]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_97_01 is the DataFrame you provided

# Set the dependent variable
y = merged_df_97_01['RET']

# Filter the gsector columns with numbers behind them
gsector_columns = [col for col in merged_df_97_01.columns if col.startswith('gsector_')]

smb_hml = ['smb', 'hml', 'mktrf']
# Set the independent variables
X = merged_df_97_01[gsector_columns + smb_hml]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.450
Method:                 Least Squares   F-statistic:                     59.96
Date:                Sat, 06 May 2023   Prob (F-statistic):          3.86e-113
Time:                        07:48:55   Log-Likelihood:                 1723.9
No. Observations:                 939   AIC:                            -3420.
Df Residuals:                     925   BIC:                            -3352.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0136      0.006      2.315   

### Regression with financial ratios and smsb_hml
regression results show that the model has an R-squared value of 0.468, indicating that approximately 46.8% of the variance in stock returns can be explained by the included variables. The adjusted R-squared value is 0.462.

Among the included variables, the variable "ln_at" (natural logarithm of total assets) has a significant positive coefficient, suggesting that larger companies, as indicated by higher total assets, tend to have higher stock returns.

The variable "return_on_equity" also has a significant positive coefficient, indicating that companies with higher return on equity tend to have higher stock returns. The variables "hml" (high-minus-low) and "mktrf" (market factor) continue to have significant positive coefficients. This suggests that a higher difference between value and growth stock returns and a positive market return are associated with higher stock returns.



In [290]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_97_01 is the DataFrame you provided

# Set the dependent variable
y = merged_df_97_01['RET']

# Filter the gsector columns with numbers behind them
# gsector_columns = [col for col in merged_df_97_01.columns if col.startswith('gsector_')]

# Set the independent variables
X = merged_df_97_01[ratios + smb_hml]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.462
Method:                 Least Squares   F-statistic:                     81.63
Date:                Sat, 06 May 2023   Prob (F-statistic):          6.42e-120
Time:                        07:48:55   Log-Likelihood:                 1733.3
No. Observations:                 939   AIC:                            -3445.
Df Residuals:                     928   BIC:                            -3391.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0018 

### All 3 together

he updated OLS regression results show that the model has an R-squared value of 0.481, indicating that approximately 48.1% of the variance in stock returns can be explained by the included variables. The adjusted R-squared value is 0.470.

Among the included variables, the variable "return_on_equity" still has a significant positive coefficient, indicating that companies with higher return on equity tend to have higher stock returns. Similarly, the variables "hml" (high-minus-low) and "mktrf" (market factor) continue to have significant positive coefficients, suggesting that a higher difference between value and growth stock returns and a positive market return are associated with higher stock returns. The g_sector_20 was found to be significant as well.

However, the variable "ln_at" (natural logarithm of total assets), which was significant in the previous model, is no longer significant in this updated model. Similarly, the variables "current_ratio" (current assets divided by current liabilities), "debt_to_equity_ratio" (total debt divided by total equity), "asset_turnover_ratio" (net sales divided by average total assets), "Debt_to_Assets_Ratio" (total debt divided by total assets), "Net_Profit_Margin" (net profit divided by net sales), "smb" (small-minus-big), and the sector-related variables do not exhibit significant relationships with stock returns.

In [291]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_97_01 is the DataFrame you provided

# Set the dependent variable
y = merged_df_97_01['RET']

# Filter the gsector columns with numbers behind them
# gsector_columns = [col for col in merged_df_97_01.columns if col.startswith('gsector_')]

# Set the independent variables
X = merged_df_97_01[ratios + smb_hml + gsector_columns]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.481
Model:                            OLS   Adj. R-squared:                  0.470
Method:                 Least Squares   F-statistic:                     42.53
Date:                Sat, 06 May 2023   Prob (F-statistic):          7.45e-116
Time:                        07:48:55   Log-Likelihood:                 1744.8
No. Observations:                 939   AIC:                            -3448.
Df Residuals:                     918   BIC:                            -3346.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0030 

# 2004-2008

The updated OLS regression results indicate that approximately 72.0% (adj R-squared) of the variance in stock returns is explained by the included variables. The significant variables in this model include ln_at (natural logarithm of assets), current_ratio, Net_Profit_Margin, smb (Small Minus Big factor), hml (High Minus Low factor), mktrf (excess market return), and the industry sector variables gsector_20.0, gsector_30.0, gsector_40.0, and gsector_55.0.

Companies with higher values for ln_at, current_ratio, Net_Profit_Margin, and gsector_55.0 (Industry Sector 55.0) tend to have higher stock returns. On the other hand, smb, hml, mktrf, and gsector_40.0 (Industry Sector 40.0) are negatively associated with stock returns, indicating that smaller market capitalization, higher book-to-market ratios, higher excess market returns, and companies in gsector_40.0 tend to have lower stock returns.

so for these sectors these ratios and similar risk exposed compnies could have been predicted for the stock returns during the crash of 2004-2008

## step 1

### regression with financial ratios analysis

The new OLS regression results indicate that the model has a relatively low R-squared value of 0.055, suggesting that only about 5.5% of the variance in stock returns is explained by the included variables. The adjusted R-squared value is also low at 0.054.

Among the included variables, the variable "ln_at" (natural logarithm of total assets) has a significant positive coefficient, indicating that companies with larger total assets tend to have higher stock returns. On the other hand, the variables "current_ratio" (current assets divided by current liabilities), "debt_to_equity_ratio" (total debt divided by total equity), and "asset_turnover_ratio" (net sales divided by average total assets) have negative coefficients, suggesting that higher current ratios, higher debt-to-equity ratios, and lower asset turnover ratios are associated with lower stock returns.

The variable "return_on_equity" has a significant positive coefficient, indicating that companies with higher return on equity tend to have higher stock returns. Similarly, the variable "Net_Profit_Margin" (net profit divided by net sales) has a significant positive coefficient, suggesting that higher net profit margins are associated with higher stock returns.

The variable "Debt_to_Assets_Ratio" (total debt divided by total assets) has a significant negative coefficient, indicating that a higher ratio of debt to assets is associated with lower stock returns.

In [292]:
ret_data_2004_08 = pd.read_csv(data_folder + 'CRSP-2004-2008.csv')

crash_data = ret_data_2004_08[(ret_data_2004_08["date"] <= '2008-11-30') & (ret_data_2004_08["date"] >= '2008-09-01')]

crash_return = crash_data.groupby('TICKER')['RET'].mean()

Compustat_2007 = pd.read_csv(data_folder +'Compustat-2007.csv')


#merging both the dataframe
merged_df_04_08 = pd.merge(Compustat_2007, crash_return, left_on='tic', right_on='TICKER')
merged_df_04_08

#log the total assets
merged_df_04_08['ln_at'] = np.log(merged_df_04_08['at'] + 1)
merged_df_04_08['ln_at']


###missing data for act
missing_data = merged_df_04_08[merged_df_04_08['act'].isnull()]
X = np.log(missing_data[['at']] + 1)
y = np.log(missing_data['act'].fillna(1))

# Filter out rows with zero or negative values
valid_idx = (X >= 0).all(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Fit the linear regression model
reg = LinearRegression().fit(X, y)

# Select the subset of the DataFrame that has missing values for 'act'
missing_data = merged_df_04_08[merged_df_04_08['act'].isnull()]
X_test = np.log(missing_data[['at']] + 1)

# Fill missing 'act' values with the predicted values
y_pred = np.exp(reg.predict(X_test))
merged_df_04_08.loc[X_test.index, 'act'] = y_pred

##missing data for lct

missing_data = merged_df_04_08[merged_df_04_08['lct'].isnull()]
X = np.log(missing_data[['lt']] + 1)
y = np.log(missing_data['lct'].fillna(1))

# Filter out rows with zero or negative values
valid_idx = (X >= 0).all(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Fit the linear regression model
reg = LinearRegression().fit(X, y)

# Select the subset of the DataFrame that has missing values for 'lct'
missing_data = merged_df_04_08[merged_df_04_08['lct'].isnull()]
X_test = np.log(missing_data[['lt']] + 1)

# Fill missing 'lct' values with the predicted values
y_pred = np.exp(reg.predict(X_test))
merged_df_04_08.loc[X_test.index, 'lct'] = y_pred


#using log for the total assets, current assets and curent liability



merged_df_04_08['ln_act'] = np.log(merged_df_04_08['act'] + 1)
merged_df_04_08['ln_act']

merged_df_04_08['ln_lct'] = np.log(merged_df_04_08['lct'] + 1)
merged_df_04_08['ln_lct']


# Calculate Current Ratio
merged_df_04_08['current_ratio'] = merged_df_04_08['act'] / merged_df_04_08['lct']

# Calculate Debt to Equity Ratio
merged_df_04_08['debt_to_equity_ratio'] = merged_df_04_08['dltt'] / (merged_df_04_08['at'] - merged_df_04_08['lt'])

# Calculate Return on Equity (ROE)
merged_df_04_08['return_on_equity'] = merged_df_04_08['ni'] / (merged_df_04_08['at'] - merged_df_04_08['lt'])

# Calculate Asset Turnover Ratio
merged_df_04_08['asset_turnover_ratio'] = merged_df_04_08['sale'] / merged_df_04_08['at']

# Debt-to-Assets Ratio
merged_df_04_08['Debt_to_Assets_Ratio'] = merged_df_04_08['dltt'] / merged_df_04_08['at']

# Net Profit Margin
merged_df_04_08['Net_Profit_Margin'] = merged_df_04_08['ni'] / merged_df_04_08['sale']



#droping NA
merged_df_04_08.dropna(subset=['current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio',  'RET'], inplace=True)



#winsorizing the data
# create a list of ratio columns to use in the regression
ratio_cols = ['current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio']

limits = [0.01, 0.01]

# loop through all columns except gvkey
for col in merged_df_04_08.columns[-10:]:
    merged_df_04_08[col] = winsorize(merged_df_04_08[col], limits)

# print the resulting DataFrame
# print(merged_df_04_08)


#regressing the ratios


# create a list of ratio columns to use in the regression
ratio_cols = ['ln_at', 'current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio', 'Debt_to_Assets_Ratio','Net_Profit_Margin' ]
# define the dependent variable
y = merged_df_04_08['RET']

# define the independent variables
X = sm.add_constant(merged_df_04_08[ratio_cols])

# fit the linear regression model
model = sm.OLS(y, X).fit()

# print the regression summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     29.95
Date:                Sat, 06 May 2023   Prob (F-statistic):           1.73e-40
Time:                        07:48:55   Log-Likelihood:                 2899.0
No. Observations:                3582   AIC:                            -5782.
Df Residuals:                    3574   BIC:                            -5732.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.1454 

In [293]:
merged_df_04_08.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3582 entries, 0 to 3588
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey                 3582 non-null   int64  
 1   datadate              3582 non-null   object 
 2   fyear                 3582 non-null   int64  
 3   indfmt                3582 non-null   object 
 4   consol                3582 non-null   object 
 5   popsrc                3582 non-null   object 
 6   datafmt               3582 non-null   object 
 7   tic                   3582 non-null   object 
 8   curcd                 3582 non-null   object 
 9   act                   3582 non-null   float64
 10  at                    3582 non-null   float64
 11  che                   3582 non-null   float64
 12  cogs                  3581 non-null   float64
 13  dltt                  3582 non-null   float64
 14  lct                   3582 non-null   float64
 15  lt                   

### for second variable, regression with smb and hml

The new OLS regression results show an improved R-squared value of 0.674, indicating that approximately 67.4% of the variance in stock returns is explained by the included variables. The adjusted R-squared value is also the same at 0.674.

The variables included in this model are "mktrf" (excess market return), "smb" (Small Minus Big factor), and "hml" (High Minus Low factor).

The constant term has a coefficient of 0.0183, which represents the expected stock return when all the independent variables are zero.

The "mktrf" variable has a significant negative coefficient of -0.1324, indicating that excess market returns have a negative impact on stock returns.

The "smb" variable also has a significant negative coefficient of -0.0227, suggesting that companies with smaller market capitalization tend to have lower stock returns compared to those with larger market capitalization.

The "hml" variable has a significant negative coefficient of -0.0127, indicating that companies with higher book-to-market ratios tend to have lower stock returns compared to those with lower book-to-market ratios.



#### cleaning and merging

In [294]:
data_folder_2 = '/content/drive/MyDrive/BA_870/Data/' 

In [295]:
risk_2004_08 = pd.read_csv(data_folder_2 + 'FF-Output-2004_08.csv')
risk_2004_08.head(5)

,Ticker,const,mktrf,smb,hml
0,A,-0.001890,1.320004,0.925827,-0.281710
1,AA,-0.007386,1.970372,-0.127755,-0.497831
2,AAC,-0.030162,1.551385,1.866894,-0.567975
3,AAI,-0.008579,-1.298654,2.380476,0.718867
4,AAME,-0.013124,1.190462,-0.641674,1.431401


In [296]:
risk_2004_08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4025 entries, 0 to 4024
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Ticker  4025 non-null   object 
 1   const   4025 non-null   float64
 2   mktrf   4025 non-null   float64
 3   smb     4025 non-null   float64
 4   hml     4025 non-null   float64
dtypes: float64(4), object(1)
memory usage: 157.4+ KB


In [297]:
merged_df_04_08 = pd.merge(merged_df_04_08, risk_2004_08, left_on='tic', right_on='Ticker')
merged_df_04_08.head(5)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,curcd,act,...,debt_to_equity_ratio,return_on_equity,asset_turnover_ratio,Debt_to_Assets_Ratio,Net_Profit_Margin,Ticker,const,mktrf,smb,hml
0,1004,2008-05-31,2007,INDL,C,D,STD,AIR,USD,783.431,...,0.867858,0.128395,1.016820,0.372918,0.054259,AIR,0.009189,0.804302,0.453706,0.541477
1,1050,2007-12-31,2007,INDL,C,D,STD,CECO,USD,67.534,...,0.108220,0.154059,2.444222,0.045880,0.026721,CECO,0.001522,1.282100,0.331407,-0.694910
2,1062,2007-11-30,2007,INDL,C,D,STD,ASA,USD,1.000,...,0.000000,0.119558,0.017782,0.000000,0.905832,ASA,0.010732,0.911218,0.617429,-0.714401
3,1072,2008-03-31,2007,INDL,C,D,STD,AVX,USD,1377.205,...,0.000000,0.081708,0.767764,0.000000,0.092309,AVX,-0.007525,0.936669,0.796042,-0.401785
4,1075,2007-12-31,2007,INDL,C,D,STD,PNW,USD,947.447,...,0.885467,0.086970,0.313386,0.278122,0.087167,PNW,-0.000079,0.539848,-0.069680,0.435324


In [298]:
merged_df_04_08.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2436 entries, 0 to 2435
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey                 2436 non-null   int64  
 1   datadate              2436 non-null   object 
 2   fyear                 2436 non-null   int64  
 3   indfmt                2436 non-null   object 
 4   consol                2436 non-null   object 
 5   popsrc                2436 non-null   object 
 6   datafmt               2436 non-null   object 
 7   tic                   2436 non-null   object 
 8   curcd                 2436 non-null   object 
 9   act                   2436 non-null   float64
 10  at                    2436 non-null   float64
 11  che                   2436 non-null   float64
 12  cogs                  2436 non-null   float64
 13  dltt                  2436 non-null   float64
 14  lct                   2436 non-null   float64
 15  lt                   

#### regression

In [299]:
import pandas as pd
import statsmodels.api as sm



# construct the independent variables using the beta coefficients and the variables in merged_df_04_08
mktrf = merged_df_04_08['mktrf']
smb = merged_df_04_08['smb']
hml = merged_df_04_08['hml']
const = merged_df_04_08['const']

# construct the dependent variable
RET = merged_df_04_08['RET']

# create a regression model with RET as the dependent variable and mktrf, smb, hml as the independent variables
X = sm.add_constant(pd.concat([mktrf, smb, hml], axis=1))
model = sm.OLS(RET, X)

# fit the model
results = model.fit()

# print the regression summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.674
Model:                            OLS   Adj. R-squared:                  0.674
Method:                 Least Squares   F-statistic:                     1678.
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        07:48:55   Log-Likelihood:                 3413.8
No. Observations:                2436   AIC:                            -6820.
Df Residuals:                    2432   BIC:                            -6796.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0183      0.003      7.108      0.0

### regression with industry effects

The regression model with sector variables has an adjusted R-squared of 0.126, indicating that the included variables explain about 12.6% of the variation in stock returns. Among the sectors, only Sector 30 (coefficient: 0.0800) and Sector 55 (coefficient: 0.1034) are statistically significant, suggesting that companies in these sectors tend to have higher stock returns. The coefficients for other sectors are not statistically significant, meaning that belonging to those sectors does not have a significant impact on stock returns.

#### cleaning and merging

In [300]:
industry_1983_87 = pd.read_csv(data_folder_2 + 'GSECTOR-1986.csv')
industry_1983_87.head(5)

,gvkey,fyear,tic,gsector
0,1003,1986,ANTQ,25.0
1,1004,1986,AIR,20.0
2,1009,1986,ABSI.1,15.0
3,1010,1986,4165A,20.0
4,1011,1986,ACSE,25.0


In [301]:
industry_1983_87.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7219 entries, 0 to 7218
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   gvkey    7219 non-null   int64  
 1   fyear    7219 non-null   int64  
 2   tic      7219 non-null   object 
 3   gsector  7210 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 225.7+ KB


In [302]:
len(industry_1983_87.gsector.unique())

12

In [303]:
merged_df_04_08 = pd.merge(merged_df_04_08, industry_1983_87, left_on='tic', right_on='tic')
merged_df_04_08.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 882 entries, 0 to 881
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               882 non-null    int64  
 1   datadate              882 non-null    object 
 2   fyear_x               882 non-null    int64  
 3   indfmt                882 non-null    object 
 4   consol                882 non-null    object 
 5   popsrc                882 non-null    object 
 6   datafmt               882 non-null    object 
 7   tic                   882 non-null    object 
 8   curcd                 882 non-null    object 
 9   act                   882 non-null    float64
 10  at                    882 non-null    float64
 11  che                   882 non-null    float64
 12  cogs                  882 non-null    float64
 13  dltt                  882 non-null    float64
 14  lct                   882 non-null    float64
 15  lt                    8

#### regression

In [304]:
merged_df_04_08.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 882 entries, 0 to 881
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               882 non-null    int64  
 1   datadate              882 non-null    object 
 2   fyear_x               882 non-null    int64  
 3   indfmt                882 non-null    object 
 4   consol                882 non-null    object 
 5   popsrc                882 non-null    object 
 6   datafmt               882 non-null    object 
 7   tic                   882 non-null    object 
 8   curcd                 882 non-null    object 
 9   act                   882 non-null    float64
 10  at                    882 non-null    float64
 11  che                   882 non-null    float64
 12  cogs                  882 non-null    float64
 13  dltt                  882 non-null    float64
 14  lct                   882 non-null    float64
 15  lt                    8

In [305]:
import pandas as pd
import statsmodels.api as sm

# create dummy variables for gsector column
gsector_dummies = pd.get_dummies(merged_df_04_08['gsector'], prefix='gsector')

# concatenate the dummy variables with the original data frame
merged_df_04_08 = pd.concat([merged_df_04_08, gsector_dummies], axis=1)

# set RET as the dependent variable
y = merged_df_04_08['RET']

merged_df_04_08 = merged_df_04_08.drop(columns=['gsector_10.0'])        #removing gsector 10
# print(merged_df_04_08.info())
# set the independent variables to the dummy variables for gsector
X = merged_df_04_08.filter(regex='^gsector_')

# add a constant to the independent variables
X = sm.add_constant(X)

# create the OLS regression model
model = sm.OLS(y, X)

# fit the model
results = model.fit()

# print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     13.71
Date:                Sat, 06 May 2023   Prob (F-statistic):           1.24e-22
Time:                        07:48:56   Log-Likelihood:                 853.22
No. Observations:                 882   AIC:                            -1684.
Df Residuals:                     871   BIC:                            -1632.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.1503      0.012    -12.475   

In [306]:
merged_df_04_08.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 882 entries, 0 to 881
Data columns (total 48 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               882 non-null    int64  
 1   datadate              882 non-null    object 
 2   fyear_x               882 non-null    int64  
 3   indfmt                882 non-null    object 
 4   consol                882 non-null    object 
 5   popsrc                882 non-null    object 
 6   datafmt               882 non-null    object 
 7   tic                   882 non-null    object 
 8   curcd                 882 non-null    object 
 9   act                   882 non-null    float64
 10  at                    882 non-null    float64
 11  che                   882 non-null    float64
 12  cogs                  882 non-null    float64
 13  dltt                  882 non-null    float64
 14  lct                   882 non-null    float64
 15  lt                    8

## step 2 

#### Regression with industry effect and financial ratios
The regression model with sector variables and additional financial ratios has an adjusted R-squared of 0.150, indicating that the included variables explain about 15% of the variation in stock returns. Several variables are statistically significant in explaining stock returns.

Among the sector variables, Sector 30 (coefficient: 0.0853) and Sector 55 (coefficient: 0.1194), sector 40 and 60 as well are statistically significant, suggesting that companies in these sectors tend to have higher stock returns. Additionally, the variable "ln_at" (coefficient: 0.0049) and "current_ratio" (coefficient: 0.0044) are statistically significant, indicating that larger asset size and higher current ratio are associated with higher stock returns.

In [307]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_04_08 is the DataFrame you provided

# Set the dependent variable
y = merged_df_04_08['RET']

# Filter the gsector columns with numbers behind them
gsector_columns = [col for col in merged_df_04_08.columns if col.startswith('gsector_')]

ratios = ['ln_at','current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio', 'Debt_to_Assets_Ratio','Net_Profit_Margin' ]
# Set the independent variables
X = merged_df_04_08[gsector_columns + ratios]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     10.11
Date:                Sat, 06 May 2023   Prob (F-statistic):           6.14e-25
Time:                        07:48:56   Log-Likelihood:                 868.79
No. Observations:                 882   AIC:                            -1702.
Df Residuals:                     864   BIC:                            -1615.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.1895 

### regression with industry effect and smb/hml
The updated OLS regression results reveal an improved R-squared value of 0.712, indicating that approximately 71.2% of the variance in stock returns is explained by the included variables. The adjusted R-squared value remains the same at 0.712.

The analysis includes several variables, namely "mktrf" (excess market return), "smb" (Small Minus Big factor), and "hml" (High Minus Low factor).

The constant term in the model has a coefficient of 0.0036, representing the expected stock return when all the independent variables are zero.

Examining the individual variables, the "mktrf" variable exhibits a significant negative coefficient of -0.1341, suggesting that excess market returns have a detrimental effect on stock returns.

Similarly, the "smb" variable demonstrates a significant negative coefficient of -0.0228, indicating that companies with smaller market capitalization tend to experience lower stock returns compared to those with larger market capitalization.

Additionally, the "hml" variable shows a significant negative coefficient of -0.0181, implying that companies with higher book-to-market ratios tend to have lower stock returns relative to those with lower book-to-market ratios.

In [308]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_04_08 is the DataFrame you provided

# Set the dependent variable
y = merged_df_04_08['RET']

# Filter the gsector columns with numbers behind them
gsector_columns = [col for col in merged_df_04_08.columns if col.startswith('gsector_')]

smb_hml = ['smb', 'hml', 'mktrf']
# Set the independent variables
X = merged_df_04_08[gsector_columns + smb_hml]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.717
Model:                            OLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                     168.8
Date:                Sat, 06 May 2023   Prob (F-statistic):          4.04e-227
Time:                        07:48:56   Log-Likelihood:                 1344.8
No. Observations:                 882   AIC:                            -2662.
Df Residuals:                     868   BIC:                            -2595.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0036      0.008      0.456   

### Regression with financial ratios and smsb_hml

The updated OLS regression results show that approximately 71.8% of the variance in stock returns is explained by the included variables. The significant variables in this model include the natural logarithm of assets (ln_at), net profit margin, smb (Small Minus Big factor), hml (High Minus Low factor), and mktrf (excess market return). Companies with higher net profit margins tend to have higher stock returns, while smaller market capitalization and higher book-to-market ratios are associated with lower stock returns.






In [309]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_04_08 is the DataFrame you provided

# Set the dependent variable
y = merged_df_04_08['RET']

# Filter the gsector columns with numbers behind them
# gsector_columns = [col for col in merged_df_04_08.columns if col.startswith('gsector_')]

# Set the independent variables
X = merged_df_04_08[ratios + smb_hml]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.718
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     221.9
Date:                Sat, 06 May 2023   Prob (F-statistic):          1.24e-231
Time:                        07:48:56   Log-Likelihood:                 1347.2
No. Observations:                 882   AIC:                            -2672.
Df Residuals:                     871   BIC:                            -2620.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0025 

### All 3 together
The updated OLS regression results indicate that approximately 72.0% (adj R-squared) of the variance in stock returns is explained by the included variables. The significant variables in this model include ln_at (natural logarithm of assets), current_ratio, Net_Profit_Margin, smb (Small Minus Big factor), hml (High Minus Low factor), mktrf (excess market return), and the industry sector variables gsector_20.0, gsector_30.0, gsector_40.0, and gsector_55.0.

Companies with higher values for ln_at, current_ratio, Net_Profit_Margin, and gsector_55.0 (Industry Sector 55.0) tend to have higher stock returns. On the other hand, smb, hml, mktrf, and gsector_40.0 (Industry Sector 40.0) are negatively associated with stock returns, indicating that smaller market capitalization, higher book-to-market ratios, higher excess market returns, and companies in gsector_40.0 tend to have lower stock returns.

The coefficients for some variables, such as debt_to_equity_ratio, return_on_equity, asset_turnover_ratio, Debt_to_Assets_Ratio, and several industry sector variables, are not statistically significant, suggesting that they do not have a significant impact on stock returns.






In [310]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_04_08 is the DataFrame you provided

# Set the dependent variable
y = merged_df_04_08['RET']

# Filter the gsector columns with numbers behind them
# gsector_columns = [col for col in merged_df_04_08.columns if col.startswith('gsector_')]

# Set the independent variables
X = merged_df_04_08[ratios + smb_hml + gsector_columns]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.726
Model:                            OLS   Adj. R-squared:                  0.720
Method:                 Least Squares   F-statistic:                     114.1
Date:                Sat, 06 May 2023   Prob (F-statistic):          6.80e-226
Time:                        07:48:56   Log-Likelihood:                 1359.8
No. Observations:                 882   AIC:                            -2678.
Df Residuals:                     861   BIC:                            -2577.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0294 

# 2016-2020

The OLS regression results show that the model explains 66.8% of the variation in stock returns (adj R-squared = 0.668). Significant variables for predicting stock returns are  current_ratio, Net_Profit_Margin, smb, hml, and mktrf and gsector25. These variables have coefficients indicating their impact on stock returns. so during the crash of 2016-2020 it could have helped to predict the return

## step 1

### regression with financial ratios analysis

regression results show that the model explains only 5.7% of the variance in stock returns, as indicated by the low R-squared value. The adjusted R-squared value is also low at 5.5%.

Among the variables included in the model, only current_ratio and Debt_to_Assets_Ratio have statistically significant coefficients. An increase in current_ratio is associated with higher stock returns, while an increase in Debt_to_Assets_Ratio is associated with lower stock returns.

In [311]:
ret_data_2016_20 = pd.read_csv(data_folder + 'CRSP-2016-2020.csv')

crash_data = ret_data_2016_20[(ret_data_2016_20["date"] <= '2020-03-31') & (ret_data_2016_20["date"] >= '2020-02-01')]

crash_return = crash_data.groupby('TICKER')['RET'].mean()

Compustat_2019 = pd.read_csv(data_folder +'Compustat-2019.csv')

#merging both the dataframe
merged_df_2016_20 = pd.merge(Compustat_2019, crash_return, left_on='tic', right_on='TICKER')
merged_df_2016_20

###missing data for act
missing_data = merged_df_2016_20[merged_df_2016_20['act'].isnull()]
X = np.log(missing_data[['at']] + 1)
y = np.log(missing_data['act'].fillna(1))

# Filter out rows with zero or negative values
valid_idx = (X >= 0).all(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Fit the linear regression model
reg = LinearRegression().fit(X, y)

# Select the subset of the DataFrame that has missing values for 'act'
missing_data = merged_df_2016_20[merged_df_2016_20['act'].isnull()]
X_test = np.log(missing_data[['at']] + 1)

# Fill missing 'act' values with the predicted values
y_pred = np.exp(reg.predict(X_test))
merged_df_2016_20.loc[X_test.index, 'act'] = y_pred

##missing data for lct

missing_data = merged_df_2016_20[merged_df_2016_20['lct'].isnull()]
X = np.log(missing_data[['lt']] + 1)
y = np.log(missing_data['lct'].fillna(1))

# Filter out rows with zero or negative values
valid_idx = (X >= 0).all(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Fit the linear regression model
reg = LinearRegression().fit(X, y)

# Select the subset of the DataFrame that has missing values for 'lct'
missing_data = merged_df_2016_20[merged_df_2016_20['lct'].isnull()]
X_test = np.log(missing_data[['lt']] + 1)

# Fill missing 'lct' values with the predicted values
y_pred = np.exp(reg.predict(X_test))
merged_df_2016_20.loc[X_test.index, 'lct'] = y_pred




#using log for the total assets, current assets and curent liability

merged_df_2016_20['ln_at'] = np.log(merged_df_2016_20['at'] + 1)
merged_df_2016_20['ln_at']

merged_df_2016_20['ln_act'] = np.log(merged_df_2016_20['act'] + 1)
merged_df_2016_20['ln_act']

merged_df_2016_20['ln_lct'] = np.log(merged_df_2016_20['lct'] + 1)
merged_df_2016_20['ln_lct']


# Calculate Current Ratio
merged_df_2016_20['current_ratio'] = merged_df_2016_20['act'] / merged_df_2016_20['lct']

# Calculate Debt to Equity Ratio
merged_df_2016_20['debt_to_equity_ratio'] = merged_df_2016_20['dltt'] / (merged_df_2016_20['at'] - merged_df_2016_20['lt'])

# Calculate Return on Equity (ROE)
merged_df_2016_20['return_on_equity'] = merged_df_2016_20['ni'] / (merged_df_2016_20['at'] - merged_df_2016_20['lt'])

# Calculate Asset Turnover Ratio
merged_df_2016_20['asset_turnover_ratio'] = merged_df_2016_20['sale'] / merged_df_2016_20['at']

# Debt-to-Assets Ratio
merged_df_2016_20['Debt_to_Assets_Ratio'] = merged_df_2016_20['dltt'] / merged_df_2016_20['at']

# Net Profit Margin
merged_df_2016_20['Net_Profit_Margin'] = merged_df_2016_20['ni'] / merged_df_2016_20['sale']




#droping NA
merged_df_2016_20.dropna(subset=['current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio',  'RET'], inplace=True)



#winsorizing the data
# create a list of ratio columns to use in the regression
ratio_cols = ['current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio']

limits = [0.01, 0.01]

# loop through all columns except gvkey
for col in merged_df_2016_20.columns[-10:]:
    merged_df_2016_20[col] = winsorize(merged_df_2016_20[col], limits)

# print the resulting DataFrame
# print(merged_df_2016_20)


#regressing the ratios


# create a list of ratio columns to use in the regression
ratio_cols = ['ln_at','current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio', 'Debt_to_Assets_Ratio','Net_Profit_Margin' ]
# define the dependent variable
y = merged_df_2016_20['RET']

# define the independent variables
X = sm.add_constant(merged_df_2016_20[ratio_cols])

# fit the linear regression model
model = sm.OLS(y, X).fit()

# print the regression summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     35.13
Date:                Sat, 06 May 2023   Prob (F-statistic):           6.38e-48
Time:                        07:48:57   Log-Likelihood:                 2790.2
No. Observations:                4086   AIC:                            -5564.
Df Residuals:                    4078   BIC:                            -5514.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.1579 

In [312]:
merged_df_2016_20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4086 entries, 0 to 4095
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey                 4086 non-null   int64  
 1   datadate              4086 non-null   object 
 2   fyear                 4086 non-null   int64  
 3   indfmt                4086 non-null   object 
 4   consol                4086 non-null   object 
 5   popsrc                4086 non-null   object 
 6   datafmt               4086 non-null   object 
 7   tic                   4086 non-null   object 
 8   curcd                 4086 non-null   object 
 9   act                   4086 non-null   float64
 10  at                    4086 non-null   float64
 11  che                   4086 non-null   float64
 12  cogs                  4086 non-null   float64
 13  dltt                  4086 non-null   float64
 14  lct                   4086 non-null   float64
 15  lt                   

### for second variable, regression with smb and hml
The regression results indicate that the model explains 56.3% of the variance in stock returns, as reflected by the R-squared value. The adjusted R-squared value is also 56.2%, suggesting a good fit for the model.

The coefficients for the independent variables, mktrf, smb, and hml, are statistically significant. A one-unit increase in mktrf is associated with a decrease in stock returns by -0.0952 units. Similarly, smb and hml have negative coefficients of -0.0176 and -0.0748, respectively, indicating that increases in these variables are linked to lower stock returns.

#### cleaning and merging

In [313]:
data_folder_2 = '/content/drive/MyDrive/BA_870/Data/' 

In [314]:
risk_2016_20 = pd.read_csv(data_folder_2 + 'FF-Output-2016_20.csv')
risk_2016_20.head(5)

,Ticker,const,mktrf,smb,hml
0,A,0.003728,1.084369,-0.257950,-0.275105
1,AADR,-0.004650,0.985970,-0.034781,-0.118292
2,AAL,-0.022520,1.530861,0.259539,0.796901
3,AAMC,0.030795,0.631664,-0.176166,0.760026
4,AAME,-0.008717,0.257933,-0.598021,0.631272


In [315]:
risk_2016_20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4824 entries, 0 to 4823
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Ticker  4824 non-null   object 
 1   const   4824 non-null   float64
 2   mktrf   4824 non-null   float64
 3   smb     4824 non-null   float64
 4   hml     4824 non-null   float64
dtypes: float64(4), object(1)
memory usage: 188.6+ KB


In [316]:
merged_df_2016_20 = pd.merge(merged_df_2016_20, risk_2016_20, left_on='tic', right_on='Ticker')
merged_df_2016_20.head(5)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,curcd,act,...,debt_to_equity_ratio,return_on_equity,asset_turnover_ratio,Debt_to_Assets_Ratio,Net_Profit_Margin,Ticker,const,mktrf,smb,hml
0,1004,2020-05-31,2019,INDL,C,D,STD,AIR,USD,1438.70,...,0.743297,0.004875,1.004954,0.322703,0.002106,AIR,0.000072,1.371459,1.212935,0.778704
1,1045,2019-12-31,2019,INDL,C,D,STD,AAL,USD,8206.00,...,-8.904612,-6.134946,0.762864,0.481290,0.036838,AAL,-0.022520,1.530861,0.259539,0.796901
2,1062,2019-11-30,2019,INDL,C,D,STD,ASA,USD,1.00,...,0.000000,0.319824,0.008273,0.000000,0.765143,ASA,0.009228,0.944891,-0.474285,-0.475522
3,1075,2019-12-31,2019,INDL,C,D,STD,PNW,USD,1030.03,...,0.879572,0.096939,0.187844,0.264320,0.155081,PNW,0.006926,0.264299,-0.444280,0.356211
4,1078,2019-12-31,2019,INDL,C,D,STD,ABT,USD,15667.00,...,0.556404,0.117792,0.469957,0.256544,0.115565,ABT,0.000801,0.963047,-0.410336,-0.632858


In [317]:
merged_df_2016_20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2953 entries, 0 to 2952
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey                 2953 non-null   int64  
 1   datadate              2953 non-null   object 
 2   fyear                 2953 non-null   int64  
 3   indfmt                2953 non-null   object 
 4   consol                2953 non-null   object 
 5   popsrc                2953 non-null   object 
 6   datafmt               2953 non-null   object 
 7   tic                   2953 non-null   object 
 8   curcd                 2953 non-null   object 
 9   act                   2953 non-null   float64
 10  at                    2953 non-null   float64
 11  che                   2953 non-null   float64
 12  cogs                  2953 non-null   float64
 13  dltt                  2953 non-null   float64
 14  lct                   2953 non-null   float64
 15  lt                   

#### regression





In [318]:
import pandas as pd
import statsmodels.api as sm



# construct the independent variables using the beta coefficients and the variables in merged_df_2016_20
mktrf = merged_df_2016_20['mktrf']
smb = merged_df_2016_20['smb']
hml = merged_df_2016_20['hml']
const = merged_df_2016_20['const']

# construct the dependent variable
RET = merged_df_2016_20['RET']

# create a regression model with RET as the dependent variable and mktrf, smb, hml as the independent variables
X = sm.add_constant(pd.concat([mktrf, smb, hml], axis=1))
model = sm.OLS(RET, X)

# fit the model
results = model.fit()

# print the regression summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     1264.
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        07:48:57   Log-Likelihood:                 3360.1
No. Observations:                2953   AIC:                            -6712.
Df Residuals:                    2949   BIC:                            -6688.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0277      0.003     -9.624      0.0

### regression with industry effects
The OLS regression results show that the model explains 23.2% of the variance in stock returns, as indicated by the R-squared value. The adjusted R-squared value is slightly lower at 22.2%, suggesting a moderate fit for the model.

The coefficients for the independent variables, gsector_15.0, gsector_20.0, gsector_25.0, gsector_30.0, gsector_35.0, gsector_40.0, gsector_45.0, gsector_50.0, gsector_55.0, and gsector_60.0, are all statistically significant.

The constant term has a coefficient of -0.2406, representing the expected stock return when all independent variables are zero.

#### cleaning and merging

In [319]:
industry_1983_87 = pd.read_csv(data_folder_2 + 'GSECTOR-1986.csv')
industry_1983_87.head(5)

,gvkey,fyear,tic,gsector
0,1003,1986,ANTQ,25.0
1,1004,1986,AIR,20.0
2,1009,1986,ABSI.1,15.0
3,1010,1986,4165A,20.0
4,1011,1986,ACSE,25.0


In [320]:
industry_1983_87.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7219 entries, 0 to 7218
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   gvkey    7219 non-null   int64  
 1   fyear    7219 non-null   int64  
 2   tic      7219 non-null   object 
 3   gsector  7210 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 225.7+ KB


In [321]:
len(industry_1983_87.gsector.unique())

12

In [322]:
merged_df_2016_20 = pd.merge(merged_df_2016_20, industry_1983_87, left_on='tic', right_on='tic')
merged_df_2016_20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 0 to 743
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               744 non-null    int64  
 1   datadate              744 non-null    object 
 2   fyear_x               744 non-null    int64  
 3   indfmt                744 non-null    object 
 4   consol                744 non-null    object 
 5   popsrc                744 non-null    object 
 6   datafmt               744 non-null    object 
 7   tic                   744 non-null    object 
 8   curcd                 744 non-null    object 
 9   act                   744 non-null    float64
 10  at                    744 non-null    float64
 11  che                   744 non-null    float64
 12  cogs                  744 non-null    float64
 13  dltt                  744 non-null    float64
 14  lct                   744 non-null    float64
 15  lt                    7

#### regression

In [323]:
merged_df_2016_20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 0 to 743
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               744 non-null    int64  
 1   datadate              744 non-null    object 
 2   fyear_x               744 non-null    int64  
 3   indfmt                744 non-null    object 
 4   consol                744 non-null    object 
 5   popsrc                744 non-null    object 
 6   datafmt               744 non-null    object 
 7   tic                   744 non-null    object 
 8   curcd                 744 non-null    object 
 9   act                   744 non-null    float64
 10  at                    744 non-null    float64
 11  che                   744 non-null    float64
 12  cogs                  744 non-null    float64
 13  dltt                  744 non-null    float64
 14  lct                   744 non-null    float64
 15  lt                    7

In [324]:
import pandas as pd
import statsmodels.api as sm

# create dummy variables for gsector column
gsector_dummies = pd.get_dummies(merged_df_2016_20['gsector'], prefix='gsector')

# concatenate the dummy variables with the original data frame
merged_df_2016_20 = pd.concat([merged_df_2016_20, gsector_dummies], axis=1)

# set RET as the dependent variable
y = merged_df_2016_20['RET']

merged_df_2016_20 = merged_df_2016_20.drop(columns=['gsector_10.0'])        #removing gsector 10
# print(merged_df_2016_20.info())
# set the independent variables to the dummy variables for gsector
X = merged_df_2016_20.filter(regex='^gsector_')

# add a constant to the independent variables
X = sm.add_constant(X)

# create the OLS regression model
model = sm.OLS(y, X)

# fit the model
results = model.fit()

# print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.232
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     22.20
Date:                Sat, 06 May 2023   Prob (F-statistic):           1.78e-36
Time:                        07:48:57   Log-Likelihood:                 706.32
No. Observations:                 744   AIC:                            -1391.
Df Residuals:                     733   BIC:                            -1340.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.2406      0.014    -17.112   

In [325]:
merged_df_2016_20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744 entries, 0 to 743
Data columns (total 48 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               744 non-null    int64  
 1   datadate              744 non-null    object 
 2   fyear_x               744 non-null    int64  
 3   indfmt                744 non-null    object 
 4   consol                744 non-null    object 
 5   popsrc                744 non-null    object 
 6   datafmt               744 non-null    object 
 7   tic                   744 non-null    object 
 8   curcd                 744 non-null    object 
 9   act                   744 non-null    float64
 10  at                    744 non-null    float64
 11  che                   744 non-null    float64
 12  cogs                  744 non-null    float64
 13  dltt                  744 non-null    float64
 14  lct                   744 non-null    float64
 15  lt                    7

## step 2 

### Regression with industry effect and financial ratios
The OLS regression results indicate that the model explains 28.2% of the variance in stock returns (R-squared). The adjusted R-squared value is slightly lower at 26.5%, suggesting a reasonable fit for the model.

Among the independent variables, the coefficients for the gsector variables (gsector_15.0, gsector_20.0, gsector_25.0, gsector_30.0, gsector_35.0, gsector_40.0, gsector_45.0, gsector_50.0, gsector_55.0, and gsector_60.0) are statistically significant. They represent different sectors, and their positive coefficients indicate that being in those sectors is associated with higher stock returns compared to the base sector.

variables such as current_ratio, return_on_equity, Debt_to_Assets_Ratio, and Net_Profit_Margin have coefficients that are statistically significant, indicating their impact on stock returns.

In [326]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_2016_20 is the DataFrame you provided

# Set the dependent variable
y = merged_df_2016_20['RET']

# Filter the gsector columns with numbers behind them
gsector_columns = [col for col in merged_df_2016_20.columns if col.startswith('gsector_')]

ratios = ['ln_at','current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio', 'Debt_to_Assets_Ratio','Net_Profit_Margin' ]
# Set the independent variables
X = merged_df_2016_20[gsector_columns + ratios]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.282
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                     16.78
Date:                Sat, 06 May 2023   Prob (F-statistic):           5.69e-42
Time:                        07:48:57   Log-Likelihood:                 731.14
No. Observations:                 744   AIC:                            -1426.
Df Residuals:                     726   BIC:                            -1343.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.2478 

### regression with industry effect and smb/hml

The OLS regression results indicate that the model explains 66.7% of the variance in stock returns (R-squared). The adjusted R-squared value is slightly lower at 66.1%, suggesting a reasonably good fit for the model.

Among the independent variables, only smb, hml, and mktrf have statistically significant coefficients. The smb variable has a coefficient of -0.0202, indicating that a higher SMB (small minus big) factor is associated with lower stock returns. The hml variable has a coefficient of -0.0694, indicating that a higher HML (high minus low) factor is associated with lower stock returns. Lastly, the mktrf variable has a coefficient of -0.1041, indicating that a higher market return factor (MKT-RF) is associated with lower stock returns.



In [327]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_2016_20 is the DataFrame you provided

# Set the dependent variable
y = merged_df_2016_20['RET']

# Filter the gsector columns with numbers behind them
gsector_columns = [col for col in merged_df_2016_20.columns if col.startswith('gsector_')]

smb_hml = ['smb', 'hml', 'mktrf']
# Set the independent variables
X = merged_df_2016_20[gsector_columns + smb_hml]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     112.7
Date:                Sat, 06 May 2023   Prob (F-statistic):          1.66e-164
Time:                        07:48:57   Log-Likelihood:                 1017.3
No. Observations:                 744   AIC:                            -2007.
Df Residuals:                     730   BIC:                            -1942.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0166      0.012     -1.389   

### Regression with financial ratios and smsb_hml

regression results, the model explains 65.7% of the stock return variation (R-squared). Significant variables for predicting stock returns include ln_at, current_ratio, debt_to_equity_ratio, asset_turnover_ratio, Debt_to_Assets_Ratio, Net_Profit_Margin, smb, hml, and mktrf. These variables have coefficients that indicate their impact on stock returns. 

In [328]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_2016_20 is the DataFrame you provided

# Set the dependent variable
y = merged_df_2016_20['RET']

# Filter the gsector columns with numbers behind them
# gsector_columns = [col for col in merged_df_2016_20.columns if col.startswith('gsector_')]

# Set the independent variables
X = merged_df_2016_20[ratios + smb_hml]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     140.6
Date:                Sat, 06 May 2023   Prob (F-statistic):          5.06e-163
Time:                        07:48:57   Log-Likelihood:                 1006.3
No. Observations:                 744   AIC:                            -1991.
Df Residuals:                     733   BIC:                            -1940.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0531 

### All 3 together
The OLS regression results show that the model explains 66.8% of the variation in stock returns (adj R-squared = 0.668). Significant variables for predicting stock returns are  current_ratio, Net_Profit_Margin, smb, hml, and mktrf and gsector25. These variables have coefficients indicating their impact on stock returns. so during the crash of 2016-2020 it could have helped to predict the return

In [329]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_2016_20 is the DataFrame you provided

# Set the dependent variable
y = merged_df_2016_20['RET']

# Filter the gsector columns with numbers behind them
# gsector_columns = [col for col in merged_df_2016_20.columns if col.startswith('gsector_')]

# Set the independent variables
X = merged_df_2016_20[ratios + smb_hml + gsector_columns]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.677
Model:                            OLS   Adj. R-squared:                  0.668
Method:                 Least Squares   F-statistic:                     75.87
Date:                Sat, 06 May 2023   Prob (F-statistic):          2.73e-162
Time:                        07:48:57   Log-Likelihood:                 1028.6
No. Observations:                 744   AIC:                            -2015.
Df Residuals:                     723   BIC:                            -1918.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0458 

# 2018-2022
In the OLS regression results, the model using the extended dataset explains 29.1% of the variation in stock returns (R-squared = 0.291). The adjusted R-squared is 27.1%. The intercept (const) is 0.0083, representing the expected stock return when all variables are zero.

Among the independent variables, smb, hml, mktrf, and asset_turnover_ratio have statistically significant coefficients, suggesting they impact stock returns. The remaining variables (ln_at, current_ratio, debt_to_equity_ratio, return_on_equity, Debt_to_Assets_Ratio, and Net_Profit_Margin) are not statistically significant.

The additional variables related to gsector (industry sector) also have mixed statistical significance. Only gsector_15, gsector_40, gsector_60 have  statistically significant coefficient, indicating they an impact on stock returns.

so for these sectors in 2018-2022 crash stock return could have been predicted by these variables or could have been the most important in predicting those returns.

## step 1

### regression with financial ratios analysis
 the model explains 6.0% of the variation in stock returns (R-squared = 0.060). The significant variables for predicting stock returns are ln_at, current_ratio, return_on_equity, asset_turnover_ratio, and Debt_to_Assets_Ratio. These variables have coefficients that indicate their impact on stock returns. However, debt_to_equity_ratio and Net_Profit_Margin are not statistically significant. The intercept (const) is -0.1126, representing the expected stock return when other variables are zero.

In [330]:
ret_data_2018_22 = pd.read_csv(data_folder + 'CRSP-2018-2022.csv')

crash_data = ret_data_2018_22[(ret_data_2018_22["date"] >= '2022-04-01') & (ret_data_2018_22["date"] <= '2022-06-30')]

crash_return = crash_data.groupby('TICKER')['RET'].mean()

Compustat_2021 = pd.read_csv(data_folder +'Compustat-2021.csv')

#merging both the dataframe
merged_df_2018_22 = pd.merge(Compustat_2021, crash_return, left_on='tic', right_on='TICKER')
merged_df_2018_22





###missing data for act
missing_data = merged_df_2018_22[merged_df_2018_22['act'].isnull()]
X = np.log(missing_data[['at']] + 1)
y = np.log(missing_data['act'].fillna(1))

# Filter out rows with zero or negative values
valid_idx = (X >= 0).all(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Fit the linear regression model
reg = LinearRegression().fit(X, y)

# Select the subset of the DataFrame that has missing values for 'act'
missing_data = merged_df_2018_22[merged_df_2018_22['act'].isnull()]
X_test = np.log(missing_data[['at']] + 1)

# Fill missing 'act' values with the predicted values
y_pred = np.exp(reg.predict(X_test))
merged_df_2018_22.loc[X_test.index, 'act'] = y_pred

##missing data for lct

missing_data = merged_df_2018_22[merged_df_2018_22['lct'].isnull()]
X = np.log(missing_data[['lt']] + 1)
y = np.log(missing_data['lct'].fillna(1))

# Filter out rows with zero or negative values
valid_idx = (X >= 0).all(axis=1)
X = X[valid_idx]
y = y[valid_idx]

# Fit the linear regression model
reg = LinearRegression().fit(X, y)

# Select the subset of the DataFrame that has missing values for 'lct'
missing_data = merged_df_2018_22[merged_df_2018_22['lct'].isnull()]
X_test = np.log(missing_data[['lt']] + 1)

# Fill missing 'lct' values with the predicted values
y_pred = np.exp(reg.predict(X_test))
merged_df_2018_22.loc[X_test.index, 'lct'] = y_pred


#using log for the total assets, current assets and curent liability

merged_df_2018_22['ln_at'] = np.log(merged_df_2018_22['at'] + 1)
merged_df_2018_22['ln_at']

merged_df_2018_22['ln_act'] = np.log(merged_df_2018_22['act'] + 1)
merged_df_2018_22['ln_act']

merged_df_2018_22['ln_lct'] = np.log(merged_df_2018_22['lct'] + 1)
merged_df_2018_22['ln_lct']



# Calculate Current Ratio
merged_df_2018_22['current_ratio'] = merged_df_2018_22['act'] / merged_df_2018_22['lct']

# Calculate Debt to Equity Ratio
merged_df_2018_22['debt_to_equity_ratio'] = merged_df_2018_22['dltt'] / (merged_df_2018_22['at'] - merged_df_2018_22['lt'])

# Calculate Return on Equity (ROE)
merged_df_2018_22['return_on_equity'] = merged_df_2018_22['ni'] / (merged_df_2018_22['at'] - merged_df_2018_22['lt'])

# Calculate Asset Turnover Ratio
merged_df_2018_22['asset_turnover_ratio'] = merged_df_2018_22['sale'] / merged_df_2018_22['at']

# Debt-to-Assets Ratio
merged_df_2018_22['Debt_to_Assets_Ratio'] = merged_df_2018_22['dltt'] / merged_df_2018_22['at']

# Net Profit Margin
merged_df_2018_22['Net_Profit_Margin'] = merged_df_2018_22['ni'] / merged_df_2018_22['sale']


#droping NA
merged_df_2018_22.dropna(subset=['current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio',  'RET'], inplace=True)



#winsorizing the data
# create a list of ratio columns to use in the regression
ratio_cols = ['current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio']

limits = [0.01, 0.01]

# loop through all columns except gvkey
for col in merged_df_2018_22.columns[-10:]:
    merged_df_2018_22[col] = winsorize(merged_df_2018_22[col], limits)

# print the resulting DataFrame
# print(merged_df_2018_22)


#regressing the ratios


# create a list of ratio columns to use in the regression
ratio_cols = ['ln_at','current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio', 'Debt_to_Assets_Ratio','Net_Profit_Margin' ]
# define the dependent variable
y = merged_df_2018_22['RET']

# define the independent variables
X = sm.add_constant(merged_df_2018_22[ratio_cols])

# fit the linear regression model
model = sm.OLS(y, X).fit()

# print the regression summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     43.48
Date:                Sat, 06 May 2023   Prob (F-statistic):           6.50e-60
Time:                        07:48:58   Log-Likelihood:                 4967.1
No. Observations:                4769   AIC:                            -9918.
Df Residuals:                    4761   BIC:                            -9866.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.1126 

In [331]:
merged_df_2018_22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4769 entries, 0 to 4781
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey                 4769 non-null   int64  
 1   datadate              4769 non-null   object 
 2   fyear                 4769 non-null   int64  
 3   indfmt                4769 non-null   object 
 4   consol                4769 non-null   object 
 5   popsrc                4769 non-null   object 
 6   datafmt               4769 non-null   object 
 7   tic                   4769 non-null   object 
 8   curcd                 4769 non-null   object 
 9   act                   4769 non-null   float64
 10  at                    4769 non-null   float64
 11  che                   4769 non-null   float64
 12  cogs                  4769 non-null   float64
 13  dltt                  4769 non-null   float64
 14  lct                   4769 non-null   float64
 15  lt                   

### for second variable, regression with smb and hml
regression results using the new data range from 2018 to 2022, the model explains 21.5% (adj r-square) of the variation in stock returns (R-squared = 0.216). The significant variables for predicting stock returns are mktrf, smb, and hml. These variables have coefficients that indicate their impact on stock returns. The intercept (const) is -0.0093, representing the expected stock return when other variables are zero. 

#### cleaning and merging

In [332]:
data_folder_2 = '/content/drive/MyDrive/BA_870/Data/' 

In [333]:
risk_2018_22 = pd.read_csv(data_folder_2 + 'FF-Output-2018-2022.csv')
risk_2018_22.head(5)

,Ticker,const,mktrf,smb,hml
0,A,0.007741,1.051265,-0.164214,-0.067821
1,AA,0.001037,2.142431,1.037943,1.601188
2,AADR,-0.008674,0.960745,-0.004855,0.126543
3,AAL,-0.023395,1.266814,0.905798,1.203338
4,AAME,0.002168,0.267055,0.518948,0.751587


In [334]:
risk_2018_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4966 entries, 0 to 4965
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Ticker  4966 non-null   object 
 1   const   4966 non-null   float64
 2   mktrf   4966 non-null   float64
 3   smb     4966 non-null   float64
 4   hml     4966 non-null   float64
dtypes: float64(4), object(1)
memory usage: 194.1+ KB


In [335]:
merged_df_2018_22 = pd.merge(merged_df_2018_22, risk_2018_22, left_on='tic', right_on='Ticker')
merged_df_2018_22.head(5)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,curcd,act,...,debt_to_equity_ratio,return_on_equity,asset_turnover_ratio,Debt_to_Assets_Ratio,Net_Profit_Margin,Ticker,const,mktrf,smb,hml
0,1004,2022-05-31,2021,INDL,C,D,STD,AIR,USD,1007.2,...,0.151087,0.076075,1.154521,0.099307,0.043311,AIR,0.001633,1.335469,0.958091,1.189697
1,1045,2021-12-31,2021,INDL,C,D,STD,AAL,USD,17336.0,...,-5.746730,0.271526,0.449576,0.634616,-0.066696,AAL,-0.023395,1.266814,0.905798,1.203338
2,1062,2021-11-30,2021,INDL,C,D,STD,ASA,USD,1.0,...,0.000000,0.038074,0.005540,0.000000,1.112973,ASA,-0.000416,1.053448,-0.721790,-0.115132
3,1075,2021-12-31,2021,INDL,C,D,STD,PNW,USD,1551.1,...,1.269150,0.102752,0.172876,0.347319,0.162657,PNW,-0.000840,0.438633,-0.509905,0.360520
4,1078,2021-12-31,2021,INDL,C,D,STD,ABT,USD,24239.0,...,0.506662,0.196286,0.572836,0.242726,0.164156,ABT,0.007250,0.714328,-0.381452,-0.229524


In [336]:
merged_df_2018_22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3017 entries, 0 to 3016
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey                 3017 non-null   int64  
 1   datadate              3017 non-null   object 
 2   fyear                 3017 non-null   int64  
 3   indfmt                3017 non-null   object 
 4   consol                3017 non-null   object 
 5   popsrc                3017 non-null   object 
 6   datafmt               3017 non-null   object 
 7   tic                   3017 non-null   object 
 8   curcd                 3017 non-null   object 
 9   act                   3017 non-null   float64
 10  at                    3017 non-null   float64
 11  che                   3017 non-null   float64
 12  cogs                  3017 non-null   float64
 13  dltt                  3017 non-null   float64
 14  lct                   3017 non-null   float64
 15  lt                   

#### regression


In [337]:
import pandas as pd
import statsmodels.api as sm



# construct the independent variables using the beta coefficients and the variables in merged_df_2018_22
mktrf = merged_df_2018_22['mktrf']
smb = merged_df_2018_22['smb']
hml = merged_df_2018_22['hml']
const = merged_df_2018_22['const']

# construct the dependent variable
RET = merged_df_2018_22['RET']

# create a regression model with RET as the dependent variable and mktrf, smb, hml as the independent variables
X = sm.add_constant(pd.concat([mktrf, smb, hml], axis=1))
model = sm.OLS(RET, X)

# fit the model
results = model.fit()

# print the regression summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.216
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     276.6
Date:                Sat, 06 May 2023   Prob (F-statistic):          1.48e-158
Time:                        07:48:59   Log-Likelihood:                 3985.1
No. Observations:                3017   AIC:                            -7962.
Df Residuals:                    3013   BIC:                            -7938.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0093      0.003     -3.717      0.0

### regression with industry effects

In the OLS regression results, using the new data, the model explains 5.7% of the variation in stock returns (R-squared = 0.057). The adjusted R-squared is 4.3%. The intercept (const) is -0.0218, representing the expected stock return when all sector variables are zero. Among the sector variables, gsector_15.0, gsector_20.0, gsector_25.0, gsector_35.0, gsector_40.0, gsector_45.0, and gsector_60.0 have statistically significant coefficients, suggesting they impact stock returns. The remaining sector variables (gsector_30.0, gsector_50.0, and gsector_55.0) are not statistically significant.

#### cleaning and merging

In [338]:
industry_1983_87 = pd.read_csv(data_folder_2 + 'GSECTOR-1986.csv')
industry_1983_87.head(5)

,gvkey,fyear,tic,gsector
0,1003,1986,ANTQ,25.0
1,1004,1986,AIR,20.0
2,1009,1986,ABSI.1,15.0
3,1010,1986,4165A,20.0
4,1011,1986,ACSE,25.0


In [339]:
industry_1983_87.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7219 entries, 0 to 7218
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   gvkey    7219 non-null   int64  
 1   fyear    7219 non-null   int64  
 2   tic      7219 non-null   object 
 3   gsector  7210 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 225.7+ KB


In [340]:
len(industry_1983_87.gsector.unique())

12

In [341]:
merged_df_2018_22 = pd.merge(merged_df_2018_22, industry_1983_87, left_on='tic', right_on='tic')
merged_df_2018_22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 709 entries, 0 to 708
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               709 non-null    int64  
 1   datadate              709 non-null    object 
 2   fyear_x               709 non-null    int64  
 3   indfmt                709 non-null    object 
 4   consol                709 non-null    object 
 5   popsrc                709 non-null    object 
 6   datafmt               709 non-null    object 
 7   tic                   709 non-null    object 
 8   curcd                 709 non-null    object 
 9   act                   709 non-null    float64
 10  at                    709 non-null    float64
 11  che                   709 non-null    float64
 12  cogs                  709 non-null    float64
 13  dltt                  709 non-null    float64
 14  lct                   709 non-null    float64
 15  lt                    7

#### regression

In [342]:
merged_df_2018_22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 709 entries, 0 to 708
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               709 non-null    int64  
 1   datadate              709 non-null    object 
 2   fyear_x               709 non-null    int64  
 3   indfmt                709 non-null    object 
 4   consol                709 non-null    object 
 5   popsrc                709 non-null    object 
 6   datafmt               709 non-null    object 
 7   tic                   709 non-null    object 
 8   curcd                 709 non-null    object 
 9   act                   709 non-null    float64
 10  at                    709 non-null    float64
 11  che                   709 non-null    float64
 12  cogs                  709 non-null    float64
 13  dltt                  709 non-null    float64
 14  lct                   709 non-null    float64
 15  lt                    7

In [343]:
import pandas as pd
import statsmodels.api as sm

# create dummy variables for gsector column
gsector_dummies = pd.get_dummies(merged_df_2018_22['gsector'], prefix='gsector')

# concatenate the dummy variables with the original data frame
merged_df_2018_22 = pd.concat([merged_df_2018_22, gsector_dummies], axis=1)

# set RET as the dependent variable
y = merged_df_2018_22['RET']

merged_df_2018_22 = merged_df_2018_22.drop(columns=['gsector_10.0'])        #removing gsector 10
# print(merged_df_2018_22.info())
# set the independent variables to the dummy variables for gsector
X = merged_df_2018_22.filter(regex='^gsector_')

# add a constant to the independent variables
X = sm.add_constant(X)

# create the OLS regression model
model = sm.OLS(y, X)

# fit the model
results = model.fit()

# print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     4.195
Date:                Sat, 06 May 2023   Prob (F-statistic):           1.14e-05
Time:                        07:48:59   Log-Likelihood:                 1020.6
No. Observations:                 709   AIC:                            -2019.
Df Residuals:                     698   BIC:                            -1969.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0218      0.009     -2.530   

In [344]:
merged_df_2018_22.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 709 entries, 0 to 708
Data columns (total 48 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gvkey_x               709 non-null    int64  
 1   datadate              709 non-null    object 
 2   fyear_x               709 non-null    int64  
 3   indfmt                709 non-null    object 
 4   consol                709 non-null    object 
 5   popsrc                709 non-null    object 
 6   datafmt               709 non-null    object 
 7   tic                   709 non-null    object 
 8   curcd                 709 non-null    object 
 9   act                   709 non-null    float64
 10  at                    709 non-null    float64
 11  che                   709 non-null    float64
 12  cogs                  709 non-null    float64
 13  dltt                  709 non-null    float64
 14  lct                   709 non-null    float64
 15  lt                    7

## step 2 

#### Regression with industry effect and financial ratios
In the OLS regression results, the model using the new data explains 9.7% of the variation in stock returns (R-squared = 0.097). The adjusted R-squared is 7.4%. The intercept (const) is -0.0343, representing the expected stock return when all variables are zero. Among the sector variables, gsector_15.0, gsector_20.0, gsector_35.0, gsector_40.0, gsector_45.0 have statistically significant coefficients, suggesting they impact stock returns. The remaining sector variables are not statistically significant.

Regarding the other variables only return_on_equity and debt to asset are statistically significant. The variables Net_Profit_Margin and asset_turnover_ratio are not statistically significant.

In [345]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_2018_22 is the DataFrame you provided

# Set the dependent variable
y = merged_df_2018_22['RET']

# Filter the gsector columns with numbers behind them
gsector_columns = [col for col in merged_df_2018_22.columns if col.startswith('gsector_')]

ratios = ['ln_at','current_ratio', 'debt_to_equity_ratio', 'return_on_equity', 'asset_turnover_ratio', 'Debt_to_Assets_Ratio','Net_Profit_Margin' ]
# Set the independent variables
X = merged_df_2018_22[gsector_columns + ratios]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     4.351
Date:                Sat, 06 May 2023   Prob (F-statistic):           1.44e-08
Time:                        07:48:59   Log-Likelihood:                 1036.0
No. Observations:                 709   AIC:                            -2036.
Df Residuals:                     691   BIC:                            -1954.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0343 

### regression with industry effect and smb/hml
 regression results, the model using the new data explains 27.6% of the variation in stock returns (R-squared = 0.276). The adjusted R-squared is 26.3%. The F-statistic is 20.39, indicating the overall significance of the model. The intercept (const) is 0.0201, representing the expected stock return when all variables are zero.

Among the sector variables, gsector_15.0, gsector_40.0, gsector_50.0, and gsector_60.0 have statistically significant coefficients, suggesting they impact stock returns. The remaining sector variables (gsector_20.0, gsector_25.0, gsector_30.0, gsector_35.0, gsector_45.0, and gsector_55.0) are not statistically significant.

Additionally, the factors smb, hml, and mktrf have statistically significant coefficients, indicating their impact on stock returns. smb (Small Minus Big) has a negative coefficient, indicating that smaller stocks tend to have lower returns compared to larger stocks. hml (High Minus Low) has a positive coefficient, indicating that value stocks tend to have higher returns compared to growth stocks. mktrf (Market Risk Premium) has a negative coefficient, suggesting that stock returns are negatively affected by market-wide fluctuations.

In [346]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_2018_22 is the DataFrame you provided

# Set the dependent variable
y = merged_df_2018_22['RET']

# Filter the gsector columns with numbers behind them
gsector_columns = [col for col in merged_df_2018_22.columns if col.startswith('gsector_')]

smb_hml = ['smb', 'hml', 'mktrf']
# Set the independent variables
X = merged_df_2018_22[gsector_columns + smb_hml]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.276
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     20.39
Date:                Sat, 06 May 2023   Prob (F-statistic):           5.22e-41
Time:                        07:48:59   Log-Likelihood:                 1114.5
No. Observations:                 709   AIC:                            -2201.
Df Residuals:                     695   BIC:                            -2137.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0201      0.010      1.946   

### Regression with financial ratios and smsb_hml
In the OLS regression results, the model using the original data explains 26.9% of the variation in stock returns (R-squared = 0.269). The adjusted R-squared is 25.9%. The intercept (const) is -0.0094, representing the expected stock return when all variables are zero.

Among the independent variables, asset_turnover_ratio, Net_Profit_Margin, smb, hml, and mktrf have statistically significant coefficients, suggesting they impact stock returns. The remaining variables (ln_at, current_ratio, debt_to_equity_ratio, return_on_equity, and Debt_to_Assets_Ratio) are not statistically significant.

The factors smb, hml, and mktrf have coefficients with the same interpretation as before. smb (Small Minus Big) has a negative coefficient, indicating that smaller stocks tend to have lower returns compared to larger stocks. hml (High Minus Low) has a positive coefficient, indicating that value stocks tend to have higher returns compared to growth stocks. mktrf (Market Risk Premium) has a negative coefficient, suggesting that stock returns are negatively affected by market-wide fluctuations.

In [347]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_2018_22 is the DataFrame you provided

# Set the dependent variable
y = merged_df_2018_22['RET']

# Filter the gsector columns with numbers behind them
# gsector_columns = [col for col in merged_df_2018_22.columns if col.startswith('gsector_')]

# Set the independent variables
X = merged_df_2018_22[ratios + smb_hml]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.269
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     25.69
Date:                Sat, 06 May 2023   Prob (F-statistic):           1.07e-41
Time:                        07:48:59   Log-Likelihood:                 1111.0
No. Observations:                 709   AIC:                            -2200.
Df Residuals:                     698   BIC:                            -2150.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0094 

### All 3 together
In the OLS regression results, the model using the extended dataset explains 29.1% of the variation in stock returns (R-squared = 0.291). The adjusted R-squared is 27.1%. The intercept (const) is 0.0083, representing the expected stock return when all variables are zero.

Among the independent variables, smb, hml, mktrf, and asset_turnover_ratio have statistically significant coefficients, suggesting they impact stock returns. The remaining variables (ln_at, current_ratio, debt_to_equity_ratio, return_on_equity, Debt_to_Assets_Ratio, and Net_Profit_Margin) are not statistically significant.

The additional variables related to gsector (industry sector) also have mixed statistical significance. Only gsector_15, gsector_40, gsector_60 have  statistically significant coefficient, indicating they an impact on stock returns.

In [348]:
import pandas as pd
import statsmodels.api as sm

# Assuming merged_df_2018_22 is the DataFrame you provided

# Set the dependent variable
y = merged_df_2018_22['RET']

# Filter the gsector columns with numbers behind them
# gsector_columns = [col for col in merged_df_2018_22.columns if col.startswith('gsector_')]

# Set the independent variables
X = merged_df_2018_22[ratios + smb_hml + gsector_columns]

# Add a constant column to the independent variables
X = sm.add_constant(X)

# Create the OLS regression model
model = sm.OLS(y, X)

# Fit the model
results = model.fit()

# Print the regression summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.271
Method:                 Least Squares   F-statistic:                     14.14
Date:                Sat, 06 May 2023   Prob (F-statistic):           1.20e-39
Time:                        07:48:59   Log-Likelihood:                 1122.0
No. Observations:                 709   AIC:                            -2202.
Df Residuals:                     688   BIC:                            -2106.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0083 